In [ ]:
from utilities_ketamine_analysis_v8 import *

In [ ]:
whos

# Filenames to analyze

In [ ]:
binPath = r'G:\My Drive\Fenton_Lab\Ketamine_analysis\Ketamine_data\Neurotar_HPC_ketamine\SPK\electrophysiology\\'
binFullPath = Path(binPath)

In [ ]:
#nt_Path for behavior
#sync_path for sync pulse
#ksort_path

dlc_path = binFullPath.parents[1]

files = dlc_path.glob('**/*.mat')

sync_path=[]
nt_path=[]

for i in sorted(files):
    
    if 'sync' in str(i): 
        print(i)
        sync_path.append(i)
        
    if 'behavior' in str(i): 
        print(i)
        nt_path.append(i)

print(nt_path[-1])

In [ ]:
sync_path

In [ ]:

#Example of how to append two files together
dlc_path = binFullPath
ksort_HPC_path=[]
ksort_PFC_path=[]

files = os.listdir(dlc_path)
for i in sorted(files):
            
    if 'HPCprobe' in str(i): 
        ksort_HPC_path.append(Path(r'G:\My Drive\Fenton_Lab\Ketamine_analysis\Ketamine_data\Neurotar_HPC_ketamine\SPK\electrophysiology\\' + i))
        print(i)
    if 'PFCprobe' in str(i): 
        ksort_PFC_path.append(Path(r'G:\My Drive\Fenton_Lab\Ketamine_analysis\Ketamine_data\Neurotar_HPC_ketamine\SPK\electrophysiology\\' + i))      
        print(i)
        
print(ksort_HPC_path)

# Behavior

### Load behavior and sync files for aligment

In [ ]:
pip install mat73

In [ ]:
print(len(nt_path),len(sync_path),len(ksort_HPC_path))

In [ ]:
import mat73
ts_ephys_aln = []
nt_sync_aln = []
camera_sync_aln = []
ioFS = []
x = []
y = []
yaw = []
speed = []
sync_bool = []

for ipath in range(0,len(nt_path)):
    
    ipathsync = sync_path[ipath]
    ipathnt = nt_path[ipath]

    print('experiment',ipathsync)
    
    # raw files for sync and behavior
    sync_extracted = mat73.loadmat(ipathsync)
    behavior = loadmat(ipathnt)
    
    if np.abs(sync_extracted['samplesSyncAP_HPCprobe'].size - sync_extracted['samplesCamera'].size) < 1000:
        
        ts_ephys_aln.append(sync_extracted['samplesSyncAP_HPCprobe'])
        nt_sync_aln.append(sync_extracted['samplesNeurotarStrobe'])
        camera_sync_aln.append(sync_extracted['samplesCamera'])
        ioFS.append(sync_extracted['ioFS'])
        
        x.append(behavior['nt_x'][0])
        y.append(behavior['nt_y'][0])
        yaw.append(behavior['nt_alpha'][0])
        speed.append(behavior['nt_speed'][0])
        
        
        sync_bool.append(True)
        
    else:
        sync_bool.append(False)


#     if 'SAL' in ipathsync:

In [ ]:
ipath

In [ ]:
print(sync_bool)
sync_cond = (np.hstack(sync_bool))
print(sync_cond)

In [ ]:
idx_sync_ok = np.flatnonzero(sync_cond == True)
idx_sync_ok.shape

In [ ]:
#Already alligned
sync_path_matched = np.array(sync_path)[idx_sync_ok]
nt_path_matched = np.array(nt_path)[idx_sync_ok]
ksort_HPC_path_matched = np.array(ksort_HPC_path)[idx_sync_ok]
ksort_PFC_path_matched = np.array(ksort_PFC_path)[idx_sync_ok]

In [ ]:
print(ksort_HPC_path_matched)

In [ ]:
%whos ndarray & list & DataFrame

In [ ]:
x_aln = []
y_aln = []
yaw_aln = []
speed_aln = []
duration = []
idx_camera = []
idx_camera_end = []
idx_nt = []
idx_nt_end = []

for ipath in range(0,len(x)):
    ipathsync = sync_path_matched[ipath]
    print('experiment',ipathsync)

    plt.plot(np.diff(ts_ephys_aln[ipath]))
    plt.plot(np.diff(nt_sync_aln[ipath]))
    plt.plot(np.diff(camera_sync_aln[ipath]))
    plt.show()

    print(camera_sync_aln[ipath])
    print(nt_sync_aln[ipath])
    
    _tmp_idx = []
    _tmp_idx = np.argwhere((camera_sync_aln[ipath] < nt_sync_aln[ipath][-1])==True)[-1]
    
    if ts_ephys_aln[ipath].size >= _tmp_idx:
        #Just to allign the camera with the sync, having the first frame of behavior and the last one
        idx_camera.append(np.argwhere((camera_sync_aln[ipath] > nt_sync_aln[ipath][0])==True)[0])
        idx_camera_end.append(np.argwhere((camera_sync_aln[ipath] < nt_sync_aln[ipath][-1])==True)[-1])
 
    if ts_ephys_aln[ipath].size < _tmp_idx:
        #Just to allign the camera with the sync, having the first frame of behavior and the last one
        idx_camera.append(np.argwhere((camera_sync_aln[ipath] > nt_sync_aln[ipath][0])==True)[0])
        idx_camera_end.append(np.array([ts_ephys_aln[ipath].size-1]).astype(int))
         
    
    print('camera',idx_camera[ipath],idx_camera_end[ipath])
    print(idx_camera_end[ipath]-idx_camera[ipath])


    print(ts_ephys_aln[ipath].size,nt_sync_aln[ipath].size,camera_sync_aln[ipath].size)


    duration.append(((ts_ephys_aln[ipath][idx_camera_end[ipath]] - ts_ephys_aln[ipath][idx_camera[ipath]]) /30000)[0])

    idx_nt.append(np.argwhere((nt_sync_aln[ipath] > camera_sync_aln[ipath][idx_camera[ipath]])==True)[0])
    idx_nt_end.append(np.argwhere((nt_sync_aln[ipath] < camera_sync_aln[ipath][idx_camera_end[ipath]])==True)[-1])
    print('nt',idx_nt[ipath],idx_nt_end[ipath])

    print(duration,(nt_sync_aln[ipath][idx_nt_end[ipath]] - nt_sync_aln[ipath][idx_nt[ipath]]) /ioFS)
    print('Deviation of',(duration[ipath] - (nt_sync_aln[ipath][idx_nt_end[ipath]] - nt_sync_aln[ipath][idx_nt[ipath]]) /ioFS[ipath]) * 1000, 'in miliseconds')
    print('Deviation at start',((camera_sync_aln[ipath][idx_camera[ipath]] - nt_sync_aln[ipath][idx_nt[ipath]])/ioFS[ipath]) * 1000,'in miliseconds')
    print('Deviation at end',((camera_sync_aln[ipath][idx_camera_end[ipath]] - nt_sync_aln[ipath][idx_nt_end[ipath]])/ioFS[ipath]) * 1000,'in miliseconds')

    
    
    ## align behavior

    x_aln.append(x[ipath][int(idx_nt[ipath][0]):int(idx_nt_end[ipath][0])])
    y_aln.append(y[ipath][int(idx_nt[ipath][0]):int(idx_nt_end[ipath][0])])
    yaw_aln.append(yaw[ipath][int(idx_nt[ipath][0]):int(idx_nt_end[ipath][0])])
    speed_aln.append(speed[ipath][int(idx_nt[ipath][0]):int(idx_nt_end[ipath][0])])

In [ ]:
## up to this point, we have identified the timestamp that aligns behavior and ephys

## x_aln, y_aln, yaw_aln, speed_aln are behavior variables already aligned to the camera frame TTLs

## to aligned the spiking activity below
## ts_ephys_aln[ipath][idx_camera[ipath]], ephys timestamp that detects the first camera frame TTL, where ipath is recording #
## ts_ephys_aln[ipath][idx_camera_end[ipath]], ephys timestamp that detects the last camera frame TTL, where ipath is recording #


### Electrophysiology - Load units - HPC probe

In [ ]:
cluster_info_all_HPC = []
df_units_all_HPC = []

cids_to_keep_HPC_all = []

channel_good_all_HPC = []

time_bin = 0.1 # in seconds

fs = 30000

spikes_binned_all_HPC = []

for ipath in range(0,len(x)):
    
    ksort_path = ksort_HPC_path_matched[ipath]
    print('experiment',ksort_path)

    cluster_info = []
    cluster_info_path = ksort_path.joinpath('cluster_info.tsv')
    cluster_group_path = ksort_path.joinpath('cluster_group.tsv')

    try:
        cluster_info = pd.read_csv(cluster_info_path, sep='\t')
    except:
        cluster_info = pd.read_csv(cluster_group_path, sep='\t')

    # for screening look only at good units 
    try:
        #cluster_info = cluster_info[cluster_info.group != 'noise']
        cluster_info = cluster_info[cluster_info.KSLabel == 'good']

    #    cluster_info = cluster_info[cluster_info.group == 'good']
    #    cluster_info = cluster_info[cluster_info.n_spikes > 100]
    #    cluster_info = cluster_info[(cluster_info.group == 'good') * (cluster_info.depth > 1200)]

    except:
        print('phy was not done')
        cluster_info = cluster_info[cluster_info.KSLabel == 'good']
        #cluster_info = cluster_info[cluster_info.KSLabel != 'noise']


    spike_times = np.load(ksort_path.joinpath('spike_times.npy'))
    spike_clusters = np.load(ksort_path.joinpath('spike_clusters.npy'))
    templates = np.load(ksort_path.joinpath('templates.npy'))
    spike_templates = np.load(ksort_path.joinpath('spike_templates.npy'))

    # cid_channels = dict(zip(cluster_info.id.values, cluster_info.ch.values))
    # cid_depth = dict(zip(cluster_info.id.values, cluster_info.depth.values))

    df_spikes = pd.DataFrame({'times':spike_times.flatten()/30000.0, 
               'unit_id':spike_clusters.flatten(), 
               'templates':spike_templates.flatten() })
    df_spikes.unit_id.unique().size
    
    
    
        # remove noise units
    try:
        kslabel_d = dict(zip(cluster_info.id, cluster_info.group))
    except:
        kslabel_d = dict(zip(cluster_info.cluster_id, cluster_info.KSLabel))

    df_spikes['kslabel'] = df_spikes.unit_id.map(kslabel_d)
    # remove noise units
    df_spikes = df_spikes.dropna()
#     df_spikes_all.append(df_spikes)
    cluster_info_all_HPC.append(cluster_info)
    print('allcells',df_spikes.unit_id.unique().size)
    
#     cids_to_keep = []
#     cids_to_keep = cluster_info[cluster_info['ch'].isin(ch_HPC_list[ipath])].cluster_id.values    
# #     print(cids_to_keep)
#     cids_to_keep_HPC_all.append(cids_to_keep)
    
    start_offset = []
    start_offset = int(ts_ephys_aln[ipath][idx_camera[ipath]])
    print(start_offset,start_offset/fs)
    
    
    ## start with neurotar light recording
    df_spk = []
    df_spk = df_spikes.copy()
    df_spk['spk'] = df_spikes['times']

    
    df_spk= df_spk[(df_spk['spk'] >= (ts_ephys_aln[ipath][idx_camera[ipath]]/fs)[0]) & (df_spk['spk'] <= (ts_ephys_aln[ipath][idx_camera_end[ipath]]/fs)[0])]
    df_spk.head()
    
    df_spk['spk'] = df_spk['times'] - (ts_ephys_aln[ipath][idx_camera[ipath]]/fs)[0]
    df_spk['spk'] 
    
        
    df_units = []
    df_units = bin_smooth_spk(df_spk, time_bin)  #0.01 is the time resolution to bin 0.01 s = 10ms
    
    _inp_l = []
#     _inp_l = df_units[df_units['cids'].isin(cids_to_keep)].pivot('cids', 'times', 'hz').values[:, :]
    _inp_l = df_units.pivot('cids', 'times', 'hz').values[:, :]

    spikes_binned_all_HPC.append(_inp_l)
    df_units_all_HPC.append(df_units)
    print('shape binned', _inp_l.shape)
        
    cids_good = []
    cids_good = df_units.cids.unique()
    print(cids_good.size)
    
    channel_good = []
    channel_good = cluster_info[cluster_info['cluster_id'].isin(cids_good)].ch.values    
    
    channel_good_all_HPC.append(channel_good)    

In [ ]:
cluster_info_all_PFC = []
df_units_all_PFC = []
depth_PFC_all = []

channel_good_all_PFC = []

fs = 30000

spikes_binned_all_PFC = []

for ipath in range(0,len(x)):
    
    ksort_path = ksort_PFC_path_matched[ipath]
    print('experiment',ksort_path)

    cluster_info = []
    cluster_info_path = ksort_path.joinpath('cluster_info.tsv')
    cluster_group_path = ksort_path.joinpath('cluster_group.tsv')

    try:
        cluster_info = pd.read_csv(cluster_info_path, sep='\t')
    except:
        cluster_info = pd.read_csv(cluster_group_path, sep='\t')

    # for screening look only at good units 
    try:
        #cluster_info = cluster_info[cluster_info.group != 'noise']
        cluster_info = cluster_info[cluster_info.KSLabel == 'good']

    #    cluster_info = cluster_info[cluster_info.group == 'good']
    #    cluster_info = cluster_info[cluster_info.n_spikes > 100]
    #    cluster_info = cluster_info[(cluster_info.group == 'good') * (cluster_info.depth > 1200)]

    except:
        print('phy was not done')
        cluster_info = cluster_info[cluster_info.KSLabel == 'good']
        #cluster_info = cluster_info[cluster_info.KSLabel != 'noise']


    spike_times = np.load(ksort_path.joinpath('spike_times.npy'))
    spike_clusters = np.load(ksort_path.joinpath('spike_clusters.npy'))
    templates = np.load(ksort_path.joinpath('templates.npy'))
    spike_templates = np.load(ksort_path.joinpath('spike_templates.npy'))

    # cid_channels = dict(zip(cluster_info.id.values, cluster_info.ch.values))
    # cid_depth = dict(zip(cluster_info.id.values, cluster_info.depth.values))

    df_spikes = pd.DataFrame({'times':spike_times.flatten()/30000.0, 
               'unit_id':spike_clusters.flatten(), 
               'templates':spike_templates.flatten() })
    df_spikes.unit_id.unique().size
    
    
    
        # remove noise units
    try:
        kslabel_d = dict(zip(cluster_info.id, cluster_info.group))
    except:
        kslabel_d = dict(zip(cluster_info.cluster_id, cluster_info.KSLabel))

    df_spikes['kslabel'] = df_spikes.unit_id.map(kslabel_d)
    # remove noise units
    df_spikes = df_spikes.dropna()
#     df_spikes_all.append(df_spikes)
    cluster_info_all_PFC.append(cluster_info)
    
    cid_depth, depth = [], []
    cid_depth = dict(zip(cluster_info.cluster_id.values, cluster_info.depth.values))
    depth = [cid_depth[p] for p in cid_depth]
    depth_PFC_all.append(depth)
    
    print(df_spikes.unit_id.unique().size)

    start_offset = []
    start_offset = int(ts_ephys_aln[ipath][idx_camera[ipath]])
    print(start_offset,start_offset/fs)
    
    
    ## start with neurotar light recording
    df_spk = []
    df_spk = df_spikes.copy()
    df_spk['spk'] = df_spikes['times']

    df_spk= df_spk[(df_spk['spk'] >= (ts_ephys_aln[ipath][idx_camera[ipath]]/fs)[0]) & (df_spk['spk'] <= (ts_ephys_aln[ipath][idx_camera_end[ipath]]/fs)[0])]
    df_spk.head()
    
    df_spk['spk'] = df_spk['times'] - (ts_ephys_aln[ipath][idx_camera[ipath]]/fs)[0]
    df_spk['spk'] 
    
    df_units = []
    df_units = bin_smooth_spk(df_spk, time_bin)  #0.01 is the time resolution to bin 0.01 s = 10ms
    
    _inp_l = []
    _inp_l = df_units.pivot('cids', 'times', 'hz').values[:, :]
       
    spikes_binned_all_PFC.append(_inp_l)
    df_units_all_PFC.append(df_units)
    print('shape binned', _inp_l.shape)
    
    spk_tmp = []
    
    cids_good = []
    cids_good = df_units.cids.unique()
    print(cids_good.size)
    
    channel_good = []
    channel_good = cluster_info[cluster_info['cluster_id'].isin(cids_good)].ch.values    
    
    channel_good_all_PFC.append(channel_good)

In [ ]:
## binned behavior in the chosen timebin ms

In [ ]:
bin_size = time_bin*100 ## samples per bin == 10 ms to 100 ms (100 samples per second that's the behavior sampRate)

beh_binned_all = []

for ipath in range(0,len(x)):
    print(ipath)
    
    behavior_con = []
    behavior_con = np.array([x_aln[ipath],y_aln[ipath],yaw_aln[ipath],speed_aln[ipath]])

    bins = behavior_con.shape[1] / bin_size
    
    behavior_tmp = []
    behavior_tmp = getMeanBinnedVector_2d(behavior_con.T, bins, return_format="array")
    print(behavior_tmp.shape)
    
    beh_binned_all.append(behavior_tmp)
    behavior_tmp = []


In [ ]:
## export to pickle aligned behavior and fine resolution spiking activity

In [ ]:
## takes a long time and create a huge file of preprocessed information

# outputList = [spikes_binned_all_HPC, spikes_binned_all_PFC, beh_binned_all,
#               cluster_info_all_HPC, df_units_all_HPC, cluster_info_all_PFC, df_units_all_PFC]

# # to save "processed" aligned data
# with open('ephys_behavior_aligned_HPC_PFC_all_100ms.file', 'wb') as f:
#     pickle.dump(outputList, f, pickle.HIGHEST_PROTOCOL)
# print('saved!')

In [ ]:
## import data

In [ ]:
# spikes_binned_all_HPC, spikes_binned_all_PFC, beh_binned_all, cluster_info_all_HPC, df_units_all_HPC, cluster_info_all_PFC, df_units_all_PFC = [],[],[],[],[], [],[]
# # load data
# with open('ephys_behavior_aligned_HPC_PFC_all_100ms.file', 'rb') as f3:
#     [spikes_binned_all_HPC, spikes_binned_all_PFC, beh_binned_all, cluster_info_all_HPC, df_units_all_HPC, cluster_info_all_PFC, df_units_all_PFC ] = pickle.load(f3)    
# print('loaded!')

# Behavior analysis

In [ ]:
#Plotting behavior, xy tracking as a function of speed

for irec in range(0,1): #len(x_aln)):
    
    ksort_path = ksort_HPC_path_matched[irec]
    print('experiment',ksort_path)
    
    
    fig = plt.figure(figsize = (6,6))
    ax = fig.add_subplot(1,1,1)
    p2 = ax.scatter(beh_binned_all[irec][0],beh_binned_all[irec][1], s=5, c= beh_binned_all[irec][3], cmap = 'viridis')
    fig.colorbar(p2, label='Speed')
#     p2 = ax.plot(beh_binned_all[irec][0],beh_binned_all[irec][1])
    
    
    ax.set_xlim(-150,150)
    ax.set_xlim(-150,150)
    ax.set_xlabel('X position')
    ax.set_ylabel('Y position')
    
    plt.show()

In [ ]:
#Plotting xy tracking as a function of heading angle

for irec in range(0,1): #len(x_aln)):
    
    ksort_path = ksort_HPC_path_matched[irec]
    print('experiment',ksort_path)
    
    
    fig = plt.figure(figsize = (6,6))
    ax = fig.add_subplot(1,1,1)
    p2 = ax.scatter(beh_binned_all[irec][0],beh_binned_all[irec][1], s=5, c= beh_binned_all[irec][2], cmap = 'hsv')
    fig.colorbar(p2, label='Heading')
    ax.set_xlim(-150,150)
    ax.set_xlim(-150,150)
    ax.set_xlabel('X position')
    ax.set_ylabel('Y position')
    
    plt.show()

In [ ]:
## example histogram of animal speed
bins = np.arange(0,10,0.1)
plt.hist(beh_binned_all[1][3], bins = bins)

In [ ]:
# mask for movement vs stationary
mask_mov_all = []
mask_sta_all = []
    
for irec in range(0,len(x_aln)):
    
    mask_mov = []
    mask_sta = []

#     size_to = []
#     size_to = np.min([spikes_binned_all_PFC[irec].shape[1],beh_binned_all[irec][0].size]) - 1
 
    mask_mov = np.zeros(beh_binned_all[irec][3].size).astype(bool)
    mask_mov[beh_binned_all[irec][3] > 10] = True
    mask_sta = np.invert(mask_mov)
    
    mask_mov_all.append(mask_mov)
    mask_sta_all.append(mask_sta)

In [ ]:
## plot firing rates @ 1000 ms = 60*30 or @ 200 ms, 30 min = 5*60*30 = 9000
sampRate = 10

# define the different injection times

baseline = 30*60*sampRate
dose1 = 60*60*sampRate
dose2 = 90*60*sampRate
dose3 = 120*60*sampRate

In [ ]:
# ids of drug for all the loaded recordings
drug_list = []
drug_list = ['SAL', 'SAL', 'RSK', 'RSK', 'RSK', 'RSK', 'SAL', 'SAL', 'RSK', 'RSK','RSK','1020',
             '1020','1020','1020','SAL','RSK', 'RSK', 'RK', 'SK', 'RK', 'SK','RK', 'SK', 'SK', 'RK',
             'CP101606', 'CP101606', 'PCP', 'PCP', 'CP101606', 'CP101606', 'PCP', 'PCP', 'CP101606', 'CP101606']

In [ ]:
len(drug_list)

In [ ]:
# ids of mouse ID for all the loaded recordings

mouse_list = []
mouse_list = ['M15', 'M16', 'M15', 'M15', 'M16', 'M16', 'M17', 'M18', 'M17', 'M18', 'M18', 'M20', 'M19', 'M21', 'M22',
             'M23', 'M23', 'M23', 'M24', 'M25', 'M26', 'M27', 'M24', 'M25', 'M27', 'M26', 'M28', 'M29', 'M30', 'M31', 'M28',
             'M29', 'M30', 'M31', 'M32', 'M33']

In [ ]:
## mean speed

In [ ]:
speed_all = []

ls_tmp = []

for irec in range(0,len(x)):
    
    speed_all.append([np.mean(beh_binned_all[irec][3][0:baseline]),
                                 np.mean(beh_binned_all[irec][3][baseline:dose1]),
                                 np.mean(beh_binned_all[irec][3][dose1:dose2]),
                                 np.mean(beh_binned_all[irec][3][dose2:dose3])])
    
    
    ## save mean firing rates in a list of pandas dataframes
    ls_tmp.append(pd.DataFrame({'drug':drug_list[irec], 'condition': ['baseline', 'dose1', 'dose2', 'dose3'],
                                'meanspeed':speed_all[irec]}))

In [ ]:
df_speed = []
df_speed = pd.concat(ls_tmp)
df_speed.shape

In [ ]:
drug_to_plot = ['SAL', 'PCP']

fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot(1,1,1)
sns.boxplot(data=df_speed[df_speed['drug'].isin(drug_to_plot)],
            x = 'condition', y = 'meanspeed', hue='drug', ax=ax)

sns.stripplot(data=df_speed[df_speed['drug'].isin(drug_to_plot)],
              x = 'condition', y = 'meanspeed', hue='drug', dodge = True, color = 'black', jitter = False,ax=ax)


plt.ylabel('Mean speed (cm/s)')
plt.xlabel('Treatment')
plt.show()

In [ ]:
## space coverage

In [ ]:
space_cov_baseline= []
space_cov_perc_baseline= []

# dose 1
space_cov_dose1= []
space_cov_perc_dose1= []

## dose 2
space_cov_dose2= []
space_cov_perc_dose2= []

# dose 3
space_cov_dose3= []
space_cov_perc_dose3 = []

ls_tmp = []

time_spent = 5
nbins = 6

for irec in range(0,len(x_aln)):
    
    ## baseline
    space_cov_baseline.append(np.histogram2d(beh_binned_all[irec][0][0:baseline],
                                             beh_binned_all[irec][1][0:baseline], bins=nbins, range = [[-150,150], [-150,150]]))
    space_cov_perc_baseline.append(np.sum(space_cov_baseline[irec][0] > time_spent) / (nbins*nbins)) # 10 seconds in each bin of a total of 6x6 bins

    # dose 1
    space_cov_dose1.append(np.histogram2d(beh_binned_all[irec][0][baseline:dose1],
                                             beh_binned_all[irec][1][baseline:dose1], bins=nbins, range = [[-150,150], [-150,150]]))
    space_cov_perc_dose1.append(np.sum(space_cov_dose1[irec][0] > time_spent) / (nbins*nbins)) # 10 seconds in each bin of a total of 6x6 bins
    
    ## dose 2
    space_cov_dose2.append(np.histogram2d(beh_binned_all[irec][0][dose1:dose2],
                                             beh_binned_all[irec][1][dose1:dose2], bins=nbins, range = [[-150,150], [-150,150]]))
    space_cov_perc_dose2.append(np.sum(space_cov_dose2[irec][0] > time_spent) / (nbins*nbins)) # 10 seconds in each bin of a total of 6x6 bins
    
    # dose 3
    space_cov_dose3.append(np.histogram2d(beh_binned_all[irec][0][dose2:dose3],
                                             beh_binned_all[irec][1][dose2:dose3], bins=nbins, range = [[-150,150], [-150,150]]))
    space_cov_perc_dose3.append(np.sum(space_cov_dose3[irec][0] > time_spent) / (nbins*nbins)) # 10 seconds in each bin of a total of 6x6 bins    
    
    ### save in dataframe
    ls_tmp.append(pd.DataFrame({'drug':drug_list[irec], 'condition': ['baseline', 'dose1', 'dose2', 'dose3'],
                                'space_cov':[space_cov_perc_baseline[irec], space_cov_perc_dose1[irec],space_cov_perc_dose2[irec],space_cov_perc_dose3[irec] ]}))    
    

In [ ]:
df_spacecov = []
df_spacecov = pd.concat(ls_tmp)
df_spacecov.head(1000)

In [ ]:
## summary boxplot of space coverage - drug comparison

In [ ]:
drug_to_plot = ['SAL', 'PCP']

fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot(1,1,1)
sns.boxplot(data=df_spacecov[df_spacecov['drug'].isin(drug_to_plot)],
            x = 'condition', y = 'space_cov', hue='drug', ax=ax)

sns.stripplot(data=df_spacecov[df_spacecov['drug'].isin(drug_to_plot)],
              x = 'condition', y = 'space_cov', hue='drug', dodge = True, color = 'black', jitter = False,ax=ax)

plt.ylim(0,1)
plt.ylabel('Space coverage [%]')
plt.xlabel('Treatment')
plt.show()

In [ ]:
## mean speed

speed_all = []

ls_tmp = []

for irec in range(0,len(x)):
    
    speed_all.append([np.mean(beh_binned_all[irec][3][0:baseline]),
                                 np.mean(beh_binned_all[irec][3][baseline:dose1]),
                                 np.mean(beh_binned_all[irec][3][dose1:dose2]),
                                 np.mean(beh_binned_all[irec][3][dose2:dose3])])
    
    
    ## save mean firing rates in a list of pandas dataframes
    ls_tmp.append(pd.DataFrame({'drug':drug_list[irec], 'condition': ['baseline', 'dose1', 'dose2', 'dose3'],
                                'meanspeed':speed_all[irec]}))

df_speed = []
df_speed = pd.concat(ls_tmp)
df_speed.shape

drug_to_plot = ['SAL', 'PCP']

fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot(1,1,1)
sns.boxplot(data=df_speed[df_speed['drug'].isin(drug_to_plot)],
            x = 'condition', y = 'meanspeed', hue='drug', ax=ax)

sns.stripplot(data=df_speed[df_speed['drug'].isin(drug_to_plot)],
              x = 'condition', y = 'meanspeed', hue='drug', dodge = True, color = 'black', jitter = False,ax=ax)


plt.ylabel('Mean speed (cm/s)')
plt.xlabel('Treatment')
plt.show()

## space coverage

space_cov_baseline= []
space_cov_perc_baseline= []

# dose 1
space_cov_dose1= []
space_cov_perc_dose1= []

## dose 2
space_cov_dose2= []
space_cov_perc_dose2= []

# dose 3
space_cov_dose3= []
space_cov_perc_dose3 = []

ls_tmp = []

time_spent = 5
nbins = 6

for irec in range(0,len(x_aln)):
    
    ## baseline
    space_cov_baseline.append(np.histogram2d(beh_binned_all[irec][0][0:baseline],
                                             beh_binned_all[irec][1][0:baseline], bins=nbins, range = [[-150,150], [-150,150]]))
    space_cov_perc_baseline.append(np.sum(space_cov_baseline[irec][0] > time_spent) / (nbins*nbins)) # 10 seconds in each bin of a total of 6x6 bins

    # dose 1
    space_cov_dose1.append(np.histogram2d(beh_binned_all[irec][0][baseline:dose1],
                                             beh_binned_all[irec][1][baseline:dose1], bins=nbins, range = [[-150,150], [-150,150]]))
    space_cov_perc_dose1.append(np.sum(space_cov_dose1[irec][0] > time_spent) / (nbins*nbins)) # 10 seconds in each bin of a total of 6x6 bins
    
    ## dose 2
    space_cov_dose2.append(np.histogram2d(beh_binned_all[irec][0][dose1:dose2],
                                             beh_binned_all[irec][1][dose1:dose2], bins=nbins, range = [[-150,150], [-150,150]]))
    space_cov_perc_dose2.append(np.sum(space_cov_dose2[irec][0] > time_spent) / (nbins*nbins)) # 10 seconds in each bin of a total of 6x6 bins
    
    # dose 3
    space_cov_dose3.append(np.histogram2d(beh_binned_all[irec][0][dose2:dose3],
                                             beh_binned_all[irec][1][dose2:dose3], bins=nbins, range = [[-150,150], [-150,150]]))
    space_cov_perc_dose3.append(np.sum(space_cov_dose3[irec][0] > time_spent) / (nbins*nbins)) # 10 seconds in each bin of a total of 6x6 bins    
    
    ### save in dataframe
    ls_tmp.append(pd.DataFrame({'drug':drug_list[irec], 'condition': ['baseline', 'dose1', 'dose2', 'dose3'],
                                'space_cov':[space_cov_perc_baseline[irec], space_cov_perc_dose1[irec],space_cov_perc_dose2[irec],space_cov_perc_dose3[irec] ]}))    
    

df_spacecov = []
df_spacecov = pd.concat(ls_tmp)
df_spacecov.head(1000)

## summary boxplot of space coverage - drug comparison

drug_to_plot = ['SAL', 'PCP']

fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot(1,1,1)
sns.boxplot(data=df_spacecov[df_spacecov['drug'].isin(drug_to_plot)],
            x = 'condition', y = 'space_cov', hue='drug', ax=ax)

sns.stripplot(data=df_spacecov[df_spacecov['drug'].isin(drug_to_plot)],
              x = 'condition', y = 'space_cov', hue='drug', dodge = True, color = 'black', jitter = False,ax=ax)

plt.ylim(0,1)
plt.ylabel('Space coverage [%]')
plt.xlabel('Treatment')
plt.show()



In [ ]:
## plot firing rate example for HPC and mPFC cells
fig = plt.figure(figsize=(12,4))
ax = fig.add_subplot(2,1,1)
sns.heatmap(spikes_binned_all_HPC[10][:,10*60*28:10*60*32], ax=ax, vmax = 10)
ax.set_xticks([])
ax.set_ylabel('Cell ID')

ax2 = fig.add_subplot(2,1,2)
sns.heatmap(spikes_binned_all_PFC[10][:,10*60*28:10*60*32], ax=ax2, vmax = 10)
ax2.set_ylabel('Cell ID')
ax2.set_xlabel('Sample')

# plt.savefig(r'G:\My Drive\Fenton_Lab\Ketamine_analysis\Manuscript\firingrate_samples' + '.png', transparent = True)
# plt.show()
# plt.clf()
# plt.close()      

In [ ]:
#Plotting behavior, xy tracking as a function of speed
fig = plt.figure(figsize = (6,6))
ax = fig.add_subplot(1,1,1)
# p2 = ax.scatter(beh_binned_all[0][0],beh_binned_all[0][1], s=5, c= beh_binned_all[0][3], cmap = 'viridis')
# fig.colorbar(p2, label='Speed')
p2 = ax.plot(beh_binned_all[0][0][:dose1],beh_binned_all[0][1][:dose1], color= 'black')

ax.set_xlim(-150,150)
ax.set_xlim(-150,150)
ax.set_xlabel('X position')
ax.set_ylabel('Y position')
ax.axis(False)
# plt.savefig(r'G:\My Drive\Fenton_Lab\Ketamine_analysis\Manuscript\behavior_samples' + '.pdf', transparent = True)
# plt.show()
# plt.clf()
# plt.close()     

In [ ]:
## firing rates per doses - analysis

In [ ]:
firing_rates_HPC_all = []
firing_rates_PFC_all = []

mask_fr_HPC_all = []

ls_tmp = []

for irec in range(0,len(x)):
    
    firing_rates_HPC_all.append([np.mean(spikes_binned_all_HPC[irec][:,0:baseline],axis=1),
                                 np.mean(spikes_binned_all_HPC[irec][:,baseline:dose1],axis=1),
                                 np.mean(spikes_binned_all_HPC[irec][:,dose1:dose2],axis=1),
                                 np.mean(spikes_binned_all_HPC[irec][:,dose2:dose3],axis=1)])
    
    firing_rates_PFC_all.append([np.mean(spikes_binned_all_PFC[irec][:,0:baseline],axis=1),
                                 np.mean(spikes_binned_all_PFC[irec][:,baseline:dose1],axis=1),
                                 np.mean(spikes_binned_all_PFC[irec][:,dose1:dose2],axis=1),
                                 np.mean(spikes_binned_all_PFC[irec][:,dose2:dose3],axis=1)])
    
    
    
#     ## save mean firing rates in a list of pandas dataframes
#     ls_tmp.append(pd.DataFrame({'drug':drug_list[irec], 'condition': ['baseline', 'dose1', 'dose2', 'dose3'],
#                                 'meanfr':np.mean(firing_rates_HPC_all[irec],axis=1), 'brain_region': 'HPC'}))

#     ls_tmp.append(pd.DataFrame({'drug':drug_list[irec], 'condition': ['baseline', 'dose1', 'dose2', 'dose3'],
#                                 'meanfr':np.mean(firing_rates_PFC_all[irec],axis=1), 'brain_region': 'PFC'}))    
    
    ## normalized by baseline
    ls_tmp.append(pd.DataFrame({'drug':drug_list[irec], 'condition': ['dose1', 'dose2', 'dose3'],
                                'meanfr':np.mean(firing_rates_HPC_all[irec],axis=1)[1:4]/np.mean(firing_rates_HPC_all[irec],axis=1)[0], 'brain_region': 'HPC'}))

    ls_tmp.append(pd.DataFrame({'drug':drug_list[irec], 'condition': ['dose1', 'dose2', 'dose3'],
                                'meanfr':np.mean(firing_rates_PFC_all[irec],axis=1)[1:4]/np.mean(firing_rates_PFC_all[irec],axis=1)[0],
                                'brain_region': 'PFC'}))        
    
    ## mean fr mask, filter cells in HPC with fr higher than 5 AP/s and lower than 0.01 AP/s
    mask_fr = np.zeros(spikes_binned_all_HPC[irec].shape[0]).astype(bool)
    test = (np.mean(spikes_binned_all_HPC[irec],axis=1) < 5) & (np.mean(spikes_binned_all_HPC[irec],axis=1) > 0.01)
    mask_fr[test] = True
    
    mask_fr_HPC_all.append(mask_fr)  

In [ ]:
df_meanfr = []
df_meanfr = pd.concat(ls_tmp)
df_meanfr.shape

In [ ]:
df_meanfr[df_meanfr['brain_region']=='HPC'].head(1000)

In [ ]:
## summary boxplot of mean firing rates - drug comparison

In [ ]:
drug_to_plot = ['SAL', '1020']

fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot(1,1,1)
sns.boxplot(data=df_meanfr[(df_meanfr['brain_region']=='HPC') & (df_meanfr['drug'].isin(drug_to_plot))],
            x = 'condition', y = 'meanfr', hue='drug', ax=ax)

sns.stripplot(data=df_meanfr[(df_meanfr['brain_region']=='HPC') & (df_meanfr['drug'].isin(drug_to_plot))],
              x = 'condition', y = 'meanfr', hue='drug', dodge = True, color = 'black', jitter = False,ax=ax)

plt.ylim(0,2)
plt.ylabel('Mean firing rate (AP/s)')
plt.xlabel('Treatment')
plt.show()

In [ ]:
## summary boxplot of mean firing rates - brain region comparison

In [ ]:
df_meanfr[df_meanfr['drug'].isin(drug_to_plot)].head(1000)

In [ ]:
np.unique(drug_list)

In [ ]:
for idrug in np.unique(drug_list):

    fig = plt.figure(figsize=(6,6))
    ax = fig.add_subplot(1,1,1)
    sns.boxplot(data=df_meanfr[df_meanfr['drug'] == idrug],
                x = 'condition', y = 'meanfr', hue='brain_region', ax=ax)

    sns.stripplot(data=df_meanfr[df_meanfr['drug'] == idrug],
                  x = 'condition', y = 'meanfr', hue='brain_region', dodge = True, color = 'black', jitter = False,ax=ax)

    plt.ylabel('Mean firing rate (AP/s)')
    plt.xlabel('Treatment')
    plt.title(idrug)
    plt.show()

In [ ]:
firing_rates_HPC_all = []
firing_rates_PFC_all = []

mask_fr_HPC_all = []

ls_tmp = []

for irec in range(0,len(x)):
    
    firing_rates_HPC_all.append([np.mean(spikes_binned_all_HPC[irec][:,0:baseline],axis=1),
                                 np.mean(spikes_binned_all_HPC[irec][:,baseline:dose1],axis=1),
                                 np.mean(spikes_binned_all_HPC[irec][:,dose1:dose2],axis=1),
                                 np.mean(spikes_binned_all_HPC[irec][:,dose2:dose3],axis=1)])
    
    firing_rates_PFC_all.append([np.mean(spikes_binned_all_PFC[irec][:,0:baseline],axis=1),
                                 np.mean(spikes_binned_all_PFC[irec][:,baseline:dose1],axis=1),
                                 np.mean(spikes_binned_all_PFC[irec][:,dose1:dose2],axis=1),
                                 np.mean(spikes_binned_all_PFC[irec][:,dose2:dose3],axis=1)])
    
    
    
    ## save mean firing rates in a list of pandas dataframes
    ls_tmp.append(pd.DataFrame({'drug':drug_list[irec], 'condition': 'baseline',
                                'meanfr':firing_rates_HPC_all[irec][0], 'brain_region': 'HPC'}))
    ls_tmp.append(pd.DataFrame({'drug':drug_list[irec], 'condition': 'dose1',
                                'meanfr':firing_rates_HPC_all[irec][1], 'brain_region': 'HPC'}))    
    ls_tmp.append(pd.DataFrame({'drug':drug_list[irec], 'condition': 'dose2',
                                'meanfr':firing_rates_HPC_all[irec][2], 'brain_region': 'HPC'}))    
    ls_tmp.append(pd.DataFrame({'drug':drug_list[irec], 'condition': 'dose3',
                                'meanfr':firing_rates_HPC_all[irec][3], 'brain_region': 'HPC'}))    
    

    ls_tmp.append(pd.DataFrame({'drug':drug_list[irec], 'condition': 'baseline',
                                'meanfr':firing_rates_PFC_all[irec][0], 'brain_region': 'PFC'}))    

    ls_tmp.append(pd.DataFrame({'drug':drug_list[irec], 'condition': 'dose1',
                                'meanfr':firing_rates_PFC_all[irec][1], 'brain_region': 'PFC'})) 
    
    ls_tmp.append(pd.DataFrame({'drug':drug_list[irec], 'condition': 'dose2',
                                'meanfr':firing_rates_PFC_all[irec][2], 'brain_region': 'PFC'}))     
    
    ls_tmp.append(pd.DataFrame({'drug':drug_list[irec], 'condition': 'dose3',
                                'meanfr':firing_rates_PFC_all[irec][3], 'brain_region': 'PFC'}))
    
    
    ## mean fr mask, filter cells in HPC with fr higher than 5 AP/s and lower than 0.01 AP/s
    mask_fr = np.zeros(spikes_binned_all_HPC[irec].shape[0]).astype(bool)
    test = (np.mean(spikes_binned_all_HPC[irec],axis=1) < 5) & (np.mean(spikes_binned_all_HPC[irec],axis=1) > 0.01)
    mask_fr[test] = True
    
    mask_fr_HPC_all.append(mask_fr)  

In [ ]:
df_meanfr = []
df_meanfr = pd.concat(ls_tmp)
df_meanfr.shape

In [ ]:
df_meanfr[df_meanfr['brain_region']=='HPC'].head(1000)

In [ ]:
## summary boxplot of mean firing rates - drug comparison

In [ ]:
drug_to_plot = ['SAL', 'PCP']

fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot(1,1,1)
sns.violinplot(data=df_meanfr[(df_meanfr['brain_region']=='HPC') & (df_meanfr['drug'].isin(drug_to_plot))],
            x = 'condition', y = 'meanfr', hue='drug',ax=ax)

sns.stripplot(data=df_meanfr[(df_meanfr['brain_region']=='HPC') & (df_meanfr['drug'].isin(drug_to_plot))],
              x = 'condition', y = 'meanfr', hue='drug', dodge = True, color = 'black',ax=ax)

plt.ylim(-1,10)
plt.ylabel('Mean firing rate (AP/s)')
plt.xlabel('Treatment')
plt.show()

In [ ]:
## summary boxplot of mean firing rates - brain region comparison

In [ ]:
df_meanfr[df_meanfr['drug'].isin(drug_to_plot)].head(1000)

In [ ]:
np.unique(drug_list)

In [ ]:
for idrug in np.unique(drug_list):

    fig = plt.figure(figsize=(6,6))
    ax = fig.add_subplot(1,1,1)
    sns.violinplot(data=df_meanfr[df_meanfr['drug'] == idrug],
                x = 'condition', y = 'meanfr', hue='brain_region', ax=ax)

    sns.stripplot(data=df_meanfr[df_meanfr['drug'] == idrug],
                  x = 'condition', y = 'meanfr', hue='brain_region', dodge = True, color = 'black',ax=ax)

    plt.ylim(-1,10)
    plt.ylabel('Mean firing rate (AP/s)')
    plt.xlabel('Treatment')
    plt.title(idrug)
    plt.show()

In [ ]:
bins_fr = np.arange(0,2,0.1)
fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot(1,1,1)
ax.hist(np.hstack(firing_rates_PFC_all[10][0]), bins = bins_fr)
ax.set_xlabel('Count')
ax.set_ylabel('Mean firing rate (AP/s)')
plt.show()

# Single Cell Analysis

In [ ]:
def place_cell_properties(spk_data, beh_data, binsxy, fsrate, shuf_state):
    from scipy.ndimage import gaussian_filter

    occ, rate_maps_sm, rate_maps_temp, p_info, coh = [], [], [], [], []
    nRepeat = 100  # number of shuffles

    # COMPUTING THE RATE MAPS
    occ = np.histogram2d(beh_data[0], beh_data[1], bins=binsxy)[0] / fsrate
    rate_maps_temp = [np.histogram2d(beh_data[0], beh_data[1], bins=binsxy, weights=s)[0]
                      / occ for s in spk_data]
    
    idx_nan = [np.isnan(p) for p in rate_maps_temp]

    rate_maps_sm = [gaussian_filter(np.nan_to_num(p), sigma=1) for p in rate_maps_temp]
    
#     for icell in range(0,len(rate_maps_temp)):
        
#         rate_maps_temp[icell][idx_nan[icell]] = np.nan
        
#     ## plot firing rate maps
#     for icell in range(0,len(rate_maps_temp)):
#         fig = plt.figure(figsize=(2, 2))
#         ax = fig.add_subplot(1, 1, 1)
#         sns.heatmap(rate_maps_temp[icell], cmap='jet', ax=ax, cbar_kws={'label': 'Normalized rate'})
#         ax.set_xlabel('X position')
#         ax.set_ylabel('Y position')
#         fname = 'Cell' + str(icell)
#         ax.set_title(fname)
#         plt.show()    

    # analyze rate maps
    p_info = [place_info_content(occ, m) for m in rate_maps_temp]
    coh = [neighbor_sum(m) for m in rate_maps_temp]

    ## pinfo and coherence
    #     print('Place info', np.hstack(p_info))
    #     print('Coherence', np.hstack(coh))

    if shuf_state == True:
        # Randomizing
        sTrackedCopy = spk_data.copy()
        sTrackedTimePairs = list(zip(*sTrackedCopy))

        pInfoShuffled = []
        cohShuffled = []
        for iRand in range(nRepeat):
            np.random.shuffle(sTrackedTimePairs)
            sShuffled = list(zip(*sTrackedTimePairs))

            RateMapsShuffled = [np.histogram2d(beh_data[0], beh_data[1], bins=binsxy, weights=s)[0]
                                / occ for s in sShuffled]
            pInfoShuffled.append([place_info_content(occ, m) for m in RateMapsShuffled])

            cohShuffled.append([neighbor_sum(m) for m in RateMapsShuffled])

        pInfoTest = []
        for p, z in zip(p_info, zip(*pInfoShuffled)):
            Out = [p, (p - np.mean(z)) / np.std(z)]
            pInfoTest.append([Out, Out[0] > 1 and Out[1] > 1.96])

        cohTest = []
        for p, z in zip(coh, zip(*cohShuffled)):
            Out = [p, (p - np.mean(z)) / np.std(z)]
            testTmp = Out[0] > 0.5 and Out[1] > 1.96
            cohTest.append([Out, testTmp])

        # determine whether cells are place cells
        PC_test = [c[0][1] > 1.96 and p[0][1] > 1.96 for c, p in zip(cohTest, pInfoTest)]
        fr_test = [(np.sum(s)/(s.size/10))<5 for s in spk_data]
        
    if shuf_state == False:
    
        PC_test = [c> 0.7 and p> 0.7 for c, p in zip(coh, p_info)]
        fr_test = [(np.sum(s)/(s.size/10))<5 for s in spk_data]

    PC_fr_test = [p*q for p,q in zip(PC_test, fr_test)]    
    print('number of PCs', np.sum(PC_fr_test), 'of', len(PC_test))

    ## plot firing rate maps
    for icell in np.hstack(np.argwhere(np.hstack(PC_fr_test) == True)):
        print('place information', p_info[icell])
        print('coherence', coh[icell])
        print('firing rate', np.sum(spk_data[icell])/(spk_data[icell].size/10))
        print('max PC rate', np.nanmax(np.nanmax(rate_maps_temp[icell])))

        fig = plt.figure(figsize=(2, 2))
        ax = fig.add_subplot(1, 1, 1)
#         sns.heatmap(rate_maps_temp_temp[icell], cmap='jet', ax=ax, cbar_kws={'label': 'Normalized rate'})
        sns.heatmap(rate_maps_sm[icell], cmap='jet', ax=ax, cbar_kws={'label': 'Normalized rate'})
        ax.set_xlabel('X position')
        ax.set_ylabel('Y position')
        fname = 'Cell' + str(icell)
        ax.set_title(fname)
        plt.show()

    return occ, rate_maps_temp, p_info, coh, PC_fr_test


In [ ]:
## spk is cell x time
## x and y positions in time
## win_legth which is the length of the path

def overdispersion(spk,xframes,yframes,iPosframes,win_length, thresh, frame, ipos_mode):

    Nbins = 16
    edgelength = 256/Nbins
    edges = np.arange(0,257,edgelength)
    sampRate = 7.5 # number of samples in a second

    occ_all = []
    rate_maps_all = []
    mean_fr_all = []
    mask_high_fr_all = []
    mask_deltaipos_all = []
    observed_rate_all = []
    expected_rate_all = []
    z_all = []
    
    binx_all = []
    biny_all = []
        
    if frame == "room":
        x,y,ipos = [],[],[]
        x = np.squeeze(xframes[0])
        y = np.squeeze(yframes[0])  
        ipos = iPosframes[0][0] 
        
    if frame == "arena":
        x,y,ipos = [],[],[]
        x = np.squeeze(xframes[1])
        y = np.squeeze(yframes[1])   
        ipos = iPosframes[0][1]      
    print('ipos_shape',ipos.shape)
    
    deltaipos = []
    deltaipos = iPosframes[0][0] - iPosframes[0][1] 
    
    # COMPUTING THE RATE MAPS
    for icell in range(0,spk.shape[0]):
        occ = []
        rate_maps_temp = []

        ## remove nans

        idx_nan = []
        idx_nan = np.invert(np.isnan(x) | np.isnan(y))
        
        mean_fr = []
        mean_fr = np.nansum(spk[icell])/(spk[icell].size/sampRate)
        

        # compute position map
        occ = np.histogram2d(x[idx_nan], y[idx_nan], bins=edges)[0]/sampRate
        # compute firing rate map normalized by occupancy
        rate_maps_temp = [np.histogram2d(x[idx_nan], y[idx_nan], bins=edges, weights=spk[icell][idx_nan])[0] / occ]

        # plot firing rate map per cell
        figure, axes = plt.subplots(figsize=(3,3))         
        sns.heatmap(rate_maps_temp[0])
        plt.show()
        
        
        ## not sure if that is real
        if mean_fr > 2:
#         if np.max(rate_maps_temp[0]) > 20 or np.max(rate_maps_temp[0]) < 10:
            print('high mean fr', mean_fr)
            print('PC peak rate',np.max(rate_maps_temp[0]))
            continue
        
        mean_fr_all.append(mean_fr)
        occ_all.append(occ)
        rate_maps_all.append(rate_maps_temp)
        
        ## compute overdispersion in windows of 5 seconds
        
        n_windows = []
        n_windows = spk[icell][idx_nan].size // win_length
        print('total number of epochs', n_windows)
        print('length of spiking in samples',spk[icell][idx_nan].size)
        spk_split = []
        
        if (spk[icell][idx_nan].size % win_length) > 0:
            spk_split = np.split(spk[icell][idx_nan][:-(spk[icell][idx_nan].size % win_length)],n_windows)
        if (spk[icell][idx_nan].size % win_length) == 0:
            spk_split = np.split(spk[icell][idx_nan],n_windows)
            
        print('spk.length',spk[icell][idx_nan].size)
        print('one path size',spk_split[0].shape)
        print('check')
        print(spk_split[0])
        print(spk[icell][idx_nan][0:30])

        if (spk[icell][idx_nan].size % win_length) > 0:

            x_split = []
            x_split = np.split(x[idx_nan][:-(spk[icell][idx_nan].size % win_length)],n_windows)
            y_split = []
            y_split = np.split(y[idx_nan][:-(spk[icell][idx_nan].size % win_length)],n_windows)   

            ## per time point compute expected rate, firing rate map sum over time

            bin_x = []
            bin_y = []       

            bin_x = [np.max([0,np.where(x_curr <= edges)[0][0]-1]) for x_curr in x[idx_nan][:-(spk[icell][idx_nan].size % win_length)]]
            bin_y = [np.max([0,np.where(y_curr <= edges)[0][0]-1]) for y_curr in y[idx_nan][:-(spk[icell][idx_nan].size % win_length)]]

        if (spk[icell][idx_nan].size % win_length) == 0:

            x_split = []
            x_split = np.split(x[idx_nan],n_windows)
            y_split = []
            y_split = np.split(y[idx_nan],n_windows)   

            ## per time point compute expected rate, firing rate map sum over time

            bin_x = []
            bin_y = []       

            bin_x = [np.max([0,np.where(x_curr <= edges)[0][0]-1]) for x_curr in x[idx_nan]]
            bin_y = [np.max([0,np.where(y_curr <= edges)[0][0]-1]) for y_curr in y[idx_nan]]
            
        ### ipos per cell
        if ipos_mode == "cell":
            if (spk[icell][idx_nan].size % win_length) > 0:
                deltaipos_split = np.split(deltaipos[icell][idx_nan][:-(spk[icell][idx_nan].size % win_length)],n_windows)
            if (spk[icell][idx_nan].size % win_length) == 0:
                deltaipos_split = np.split(deltaipos[icell][idx_nan],n_windows)        
 
        ### ipos as a population
        if ipos_mode == "population":
            if (spk[icell][idx_nan].size % win_length) > 0:
                deltaipos_split = np.split(np.nansum(deltaipos,axis=0)[idx_nan][:-(spk[icell][idx_nan].size % win_length)],n_windows)
            if (spk[icell][idx_nan].size % win_length) == 0:
                deltaipos_split = np.split(np.nansum(deltaipos,axis=0)[idx_nan],n_windows)        
        
        expected_timeseries = []
        expected_timeseries = [rate_maps_temp[0][bx,by]/sampRate  for bx,by in zip(bin_x,bin_y)]
#         expected_timeseries = [rate_maps_temp[0][bx,by]/sampRate  for bx,by in zip(bin_x,bin_y)]

        expected_timeseries_split = []        
        expected_timeseries_split = np.split(np.concatenate(expected_timeseries, axis=None),n_windows)   

        observed_rate, expected_rate = [], []

        # compute observed rate in the epoch
        observed_rate = [np.sum(p) for p in spk_split]

        #compute expected rate in the epoch from mean firing rate map
        expected_rate = [np.sum(p) for p in expected_timeseries_split]

        mask_highfr = []
        mask_highfr = [s > mean_fr*thresh for s in expected_rate]
        
        mask_ipos = []
        mask_ipos = [np.nanmean(s)>0 for s in deltaipos_split]
        
        mask_deltaipos_all.append(mask_ipos) ## positive means room frame

        z = []
        z = [(o-e)/np.sqrt(e) for o,e in zip(observed_rate,expected_rate)]

        mask_high_fr_all.append(mask_highfr)
        observed_rate_all.append(observed_rate)
        expected_rate_all.append(expected_rate)
        z_all.append(z)
        
        binx_all.append(bin_x)
        biny_all.append(bin_y)
        
        ## plot expected and observed rate
        
        fig = plt.figure(figsize=(8,8))
        ax = fig.add_subplot(1,1,1)
        win = np.arange(0,n_windows,1)
        ax.scatter(win, np.hstack(observed_rate)/(win_length/sampRate), color = 'black')
        ax.scatter(win, np.hstack(expected_rate)/(win_length/sampRate), color = 'red')
        ax.plot(mean_fr*np.ones(300), '--', color = 'gray')

        ax.set_ylabel('spikes per second')
        ax.set_xlabel('Path #')
        plt.show()
        
        ## plot z
        
        fig = plt.figure(figsize=(6,4))
        ax = fig.add_subplot(1,1,1)
        win = np.arange(0,n_windows,1)
        ax.scatter(win, z, color = 'black')
        ax.set_ylabel('Z')
        ax.set_xlabel('Path #')
        plt.show()        
        
    x, y, spk = [], [], []
        
    return occ_all, rate_maps_all, mean_fr_all, mask_high_fr_all,mask_deltaipos_all, observed_rate_all,expected_rate_all,z_all, binx_all, biny_all

## PC analysis in HPC cells

In [ ]:
bins_position = 20
ls_tmp = []
fr_maps_all = []
PC_test_b_all = []

for irec in range(0,len(x)):
    
#     if (drug_list[irec] == 'SAL') or (drug_list[irec] == 'PCP') or (drug_list[irec] == 'RSK'):
#     if (mouse_list[irec] == 'M15'):


    ksort_path = ksort_HPC_path_matched[irec]
    print('experiment',ksort_path)

    size_to = []
    size_to = np.min([spikes_binned_all_HPC[irec].shape[1],beh_binned_all[irec][0].size]) - 1



    # baseline
    occ_b, rate_maps_temp_b, p_info_b, coh_b, PC_test_b = [], [],[],[],[]   
    occ_b, rate_maps_temp_b, p_info_b, coh_b, PC_test_b = place_cell_properties(spikes_binned_all_HPC[irec][:,:baseline][:,mask_mov_all[irec][:baseline]],
                                                                      np.array([beh_binned_all[irec][0][:baseline][mask_mov_all[irec][:baseline]],
                                                                                beh_binned_all[irec][1][:baseline][mask_mov_all[irec][:baseline]]]),
                                                                                bins_position,10,False)
    
    PC_test_b_all.append(PC_test_b)

        
#         # dose 1
#         occ_d1, rate_maps_temp_d1, p_info_d1, coh_d1, PC_test_d1 = [], [],[],[],[]   
#         occ_d1, rate_maps_temp_d1, p_info_d1, coh_d1, PC_test_d1 = place_cell_properties(spikes_binned_all_HPC[irec][:,baseline:dose1],
#                                                                           np.array([beh_binned_all[irec][0][baseline:dose1],
#                                                                                     beh_binned_all[irec][1][baseline:dose1]]),
#                                                                                          bins_position,10, True)
        
        
#         # dose 2
#         occ_d2, rate_maps_temp_d2, p_info_d2, coh_d2, PC_test_d2 = [], [],[],[],[]   
#         occ_d2, rate_maps_temp_d2, p_info_d2, coh_d2, PC_test_d2 = place_cell_properties(spikes_binned_all_HPC[irec][:,dose1:dose2],
#                                                                           np.array([beh_binned_all[irec][0][dose1:dose2],
#                                                                                     beh_binned_all[irec][1][dose1:dose2]]),
#                                                                                          bins_position,10, True)
        
        
#         # dose 3
#         occ_d3, rate_maps_temp_d3, p_info_d3, coh_d3, PC_test_d3 = [], [],[],[],[]   
#         occ_d3, rate_maps_temp_d3, p_info_d3, coh_d3, PC_test_d3 = place_cell_properties(spikes_binned_all_HPC[irec][:,dose2:size_to],
#                                                                           np.array([beh_binned_all[irec][0][dose2:size_to],
#                                                                                     beh_binned_all[irec][1][dose2:size_to]]),
#                                                                                          bins_position,10, True)
        
        
#     ls_tmp.append(pd.DataFrame({'mouse': mouse_list[irec], 'drug': drug_list[irec], 'condition': 'baseline',
#                                    'cellid': np.arange(0,len(p_info_b)),'place_info': p_info_b, 'coherence': coh_b, 'PC_test': PC_test_b}))
        
#         ls_tmp.append(pd.DataFrame({'mouse': mouse_list[irec], 'drug': drug_list[irec], 'condition': 'dose1',
#                                    'cellid': np.arange(0,len(p_info_b)),'place_info': p_info_d1, 'coherence': coh_d1, 'PC_test': PC_test_d1}))
        
#         ls_tmp.append(pd.DataFrame({'mouse': mouse_list[irec], 'drug': drug_list[irec], 'condition': 'dose2',
#                                    'cellid': np.arange(0,len(p_info_b)),'place_info': p_info_d2, 'coherence': coh_d2, 'PC_test': PC_test_d2}))
        
#         ls_tmp.append(pd.DataFrame({'mouse': mouse_list[irec], 'drug': drug_list[irec], 'condition': 'dose3',
#                                    'cellid': np.arange(0,len(p_info_b)),'place_info': p_info_d3, 'coherence': coh_d3, 'PC_test': PC_test_d3}))      
        
        
        
#         fr_maps_all.append([rate_maps_temp_b, rate_maps_temp_d1, rate_maps_temp_d2, rate_maps_temp_d3])

## PC analysis in PFC cells

In [ ]:
bins_position = 20
ls_tmp = []
fr_maps_all = []
PC_test_b_all_PFC = []

for irec in range(0,len(x)):
    
#     if (drug_list[irec] == 'SAL') or (drug_list[irec] == 'PCP') or (drug_list[irec] == 'RSK'):
#     if (mouse_list[irec] == 'M15'):


    ksort_path = ksort_HPC_path_matched[irec]
    print('experiment',ksort_path)

    size_to = []
    size_to = np.min([spikes_binned_all_PFC[irec].shape[1],beh_binned_all[irec][0].size]) - 1



    # baseline
    occ_b, rate_maps_temp_b, p_info_b, coh_b, PC_test_b = [], [],[],[],[]   
    occ_b, rate_maps_temp_b, p_info_b, coh_b, PC_test_b = place_cell_properties(spikes_binned_all_PFC[irec][:,0:baseline][:,mask_mov_all[irec][0:baseline]],
                                                                      np.array([beh_binned_all[irec][0][0:baseline][mask_mov_all[irec][0:baseline]],
                                                                                beh_binned_all[irec][1][0:baseline][mask_mov_all[irec][0:baseline]]]),
                                                                                bins_position,10,False)
    
    PC_test_b_all_PFC.append(PC_test_b)

        
#         # dose 1
#         occ_d1, rate_maps_temp_d1, p_info_d1, coh_d1, PC_test_d1 = [], [],[],[],[]   
#         occ_d1, rate_maps_temp_d1, p_info_d1, coh_d1, PC_test_d1 = place_cell_properties(spikes_binned_all_HPC[irec][:,baseline:dose1],
#                                                                           np.array([beh_binned_all[irec][0][baseline:dose1],
#                                                                                     beh_binned_all[irec][1][baseline:dose1]]),
#                                                                                          bins_position,10, True)
        
        
#         # dose 2
#         occ_d2, rate_maps_temp_d2, p_info_d2, coh_d2, PC_test_d2 = [], [],[],[],[]   
#         occ_d2, rate_maps_temp_d2, p_info_d2, coh_d2, PC_test_d2 = place_cell_properties(spikes_binned_all_HPC[irec][:,dose1:dose2],
#                                                                           np.array([beh_binned_all[irec][0][dose1:dose2],
#                                                                                     beh_binned_all[irec][1][dose1:dose2]]),
#                                                                                          bins_position,10, True)
        
        
#         # dose 3
#         occ_d3, rate_maps_temp_d3, p_info_d3, coh_d3, PC_test_d3 = [], [],[],[],[]   
#         occ_d3, rate_maps_temp_d3, p_info_d3, coh_d3, PC_test_d3 = place_cell_properties(spikes_binned_all_HPC[irec][:,dose2:size_to],
#                                                                           np.array([beh_binned_all[irec][0][dose2:size_to],
#                                                                                     beh_binned_all[irec][1][dose2:size_to]]),
#                                                                                          bins_position,10, True)
        
        
#     ls_tmp.append(pd.DataFrame({'mouse': mouse_list[irec], 'drug': drug_list[irec], 'condition': 'baseline',
#                                    'cellid': np.arange(0,len(p_info_b)),'place_info': p_info_b, 'coherence': coh_b, 'PC_test': PC_test_b}))
        
#         ls_tmp.append(pd.DataFrame({'mouse': mouse_list[irec], 'drug': drug_list[irec], 'condition': 'dose1',
#                                    'cellid': np.arange(0,len(p_info_b)),'place_info': p_info_d1, 'coherence': coh_d1, 'PC_test': PC_test_d1}))
        
#         ls_tmp.append(pd.DataFrame({'mouse': mouse_list[irec], 'drug': drug_list[irec], 'condition': 'dose2',
#                                    'cellid': np.arange(0,len(p_info_b)),'place_info': p_info_d2, 'coherence': coh_d2, 'PC_test': PC_test_d2}))
        
#         ls_tmp.append(pd.DataFrame({'mouse': mouse_list[irec], 'drug': drug_list[irec], 'condition': 'dose3',
#                                    'cellid': np.arange(0,len(p_info_b)),'place_info': p_info_d3, 'coherence': coh_d3, 'PC_test': PC_test_d3}))      
        
        
        
#         fr_maps_all.append([rate_maps_temp_b, rate_maps_temp_d1, rate_maps_temp_d2, rate_maps_temp_d3])

In [ ]:
def normalizer(a):
    out = []
    if (np.nanmax(a) - np.nanmin(a)) == 0:
        out = (a-np.nanmin(a))
    if (np.nanmax(a) - np.nanmin(a)) != 0:
        out = (a-np.nanmin(a))/(np.nanmax(a) - np.nanmin(a))
        
    return out

In [ ]:
def fr_map_ang(spk_data, beh_data, binsxy, fsrate):
    from scipy.ndimage import gaussian_filter
    
    occ, rate_maps_temp= [], []
    
    Xcenter = beh_data[0] - (np.nanmax(beh_data[0]) + np.nanmin(beh_data[1])) / 2
    Ycenter = beh_data[1] - (np.nanmax(beh_data[1]) + np.nanmin(beh_data[0])) / 2
    
    alpha = np.arctan2(Ycenter, Xcenter)

    L_occ = np.histogram(alpha, bins=binsxy)[0] /fsrate
    LinMapsTemp_temp = [np.histogram(alpha, bins=binsxy, weights=s)[0] / L_occ
         for s in spk_data]
    
    LinMapsTemp = [normalizer(gaussian_filter(np.nan_to_num(p), sigma=1)) for p in LinMapsTemp_temp]
    

    return LinMapsTemp

In [ ]:
bins_position = 30
fr_maps_all = []

idx_plot = 0

PCs_b_all_sal = []
PCs_d1_all_sal = []

for irec in range(0,len(x)):
    

    ksort_path = ksort_HPC_path_matched[irec]
    print('experiment',ksort_path)

    size_to = []
    size_to = np.min([spikes_binned_all_HPC[irec].shape[1],beh_binned_all[irec][0].size]) - 1



    # baseline
    rate_maps_temp_b= []
    rate_maps_temp_b= fr_map_ang(spikes_binned_all_HPC[irec][:,0:baseline][:,mask_mov_all[irec][0:baseline]],
                                                                      np.array([beh_binned_all[irec][0][0:baseline][mask_mov_all[irec][0:baseline]],
                                                                                beh_binned_all[irec][1][0:baseline][mask_mov_all[irec][0:baseline]]]),
                                                                                bins_position,10)

    # baseline
    rate_maps_temp_d1= []
    rate_maps_temp_d1= fr_map_ang(spikes_binned_all_HPC[irec][:,baseline:dose1][:,mask_mov_all[irec][baseline:dose1]],
                                                                      np.array([beh_binned_all[irec][0][baseline:dose1][mask_mov_all[irec][baseline:dose1]],
                                                                                beh_binned_all[irec][1][baseline:dose1][mask_mov_all[irec][baseline:dose1]]]),
                                  bins_position,10) 
#     # baseline
#     rate_maps_temp_d1= []
#     rate_maps_temp_d1= fr_map_ang(spikes_binned_all_HPC[irec][:,baseline:size_to][:,mask_mov_all[irec][baseline:size_to]],
#                                                                       np.array([beh_binned_all[irec][0][baseline:size_to][mask_mov_all[irec][baseline:size_to]],
#                                                                                 beh_binned_all[irec][1][baseline:size_to][mask_mov_all[irec][baseline:size_to]]]),
#                                                                                 bins_position,10)   
    if (drug_list[irec] == 'SAL'):
        
            
        fig = plt.figure(figsize=(10,10))
        ax = fig.add_subplot(1,2,1)

        idx_sort = np.nanargmax(np.vstack(rate_maps_temp_b)[PC_test_b_all[irec],:].T,axis=0)
        ax.imshow(((np.vstack(rate_maps_temp_b)[PC_test_b_all[irec],:]).T[:,np.argsort(idx_sort)]).T)
        ax.set_title('Baseline')
        ax.set_xlabel('Angle')
        ax.set_ylabel('Cell ID')

        ax2 = fig.add_subplot(1,2,2)
        ax2.imshow(((np.vstack(rate_maps_temp_d1)[PC_test_b_all[irec],:]).T[:,np.argsort(idx_sort)]).T)    
        ax2.axis(False)
#         plt.show()    

#         fig = plt.figure(figsize=(5,20))
#         ax = fig.add_subplot(1,2,1)

#         idx_sort = np.nanargmax(np.vstack(rate_maps_temp_d1)[PC_test_b_all[irec],:].T,axis=0)
#         ax.imshow(((np.vstack(rate_maps_temp_b)[PC_test_b_all[irec],:]).T[:,np.argsort(idx_sort)]).T)
#         ax.set_title('Baseline')
#         ax.set_xlabel('Angle')
#         ax.set_ylabel('Cell ID')

#         ax2 = fig.add_subplot(1,2,2)
#         ax2.imshow(((np.vstack(rate_maps_temp_d1)[PC_test_b_all[irec],:]).T[:,np.argsort(idx_sort)]).T)    
#         ax2.axis(False)
        
#         plt.tight_layout()
#         plt.savefig(r'G:\My Drive\Fenton_Lab\Ketamine_analysis\Manuscript\PC_linearized_HPC_SAL' + str(idx_plot)  + '.pdf', transparent = True)
#         plt.show()
#         plt.clf()
#         plt.close()  
        
        idx_plot = idx_plot +1
        
        PCs_b_all_sal.append((np.vstack(rate_maps_temp_b)[PC_test_b_all[irec],:]).T)
        PCs_d1_all_sal.append((np.vstack(rate_maps_temp_d1)[PC_test_b_all[irec],:]).T)


In [ ]:
## plotting all PCs during baseline and dose 1

fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot(1,2,1)

idx_sort = np.nanargmax(np.hstack(PCs_b_all_sal),axis=0)


ax.imshow(np.hstack(PCs_b_all_sal)[:,np.argsort(idx_sort)].T)
ax.set_title('Saline')
ax.set_xlabel('Angle')
ax.set_ylabel('Cell ID')

ax2 = fig.add_subplot(1,2,2)
ax2.imshow(np.hstack(PCs_d1_all_sal)[:,np.argsort(idx_sort)].T)    
ax2.set_title('Saline')
ax2.axis(False)
# plt.savefig(r'G:\My Drive\Fenton_Lab\Ketamine_analysis\Manuscript\PC_linearized_HPC_SAL_summary' + '.pdf', transparent = True)
# plt.show()
# plt.clf()
# plt.close()  

## map correlation

map_corr_HPC = []
map_corr_HPC = [pearsonr(p,q).statistic for p,q in zip(np.hstack(PCs_b_all_sal).T,np.hstack(PCs_d1_all_sal).T)]

map_corr_HPC_shuf = []
for ishuf in range(0,100):
    PC_b_shuf = np.hstack(PCs_b_all_sal).T.copy()
    np.random.shuffle(PC_b_shuf)

    map_corr_HPC_shuf_tmp = []
    map_corr_HPC_shuf_tmp = [pearsonr(p,q).statistic for p,q in zip(np.hstack(PCs_b_all_sal).T,PC_b_shuf)]
    map_corr_HPC_shuf.append(np.hstack(map_corr_HPC_shuf_tmp))

print('mean map corr',np.nanmean(np.hstack(map_corr_HPC)))
print('mean map corr_shuf',np.nanmean(np.concatenate(map_corr_HPC_shuf,axis=None)))


## histogram

bins = np.arange(-1,1,0.1)
fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot(1,1,1)
sns.histplot(np.hstack(map_corr_HPC), bins=bins, color ='orange', alpha= 0.3, stat = 'probability')
sns.histplot(np.hstack(map_corr_HPC_shuf), bins=bins, color = 'black',alpha = 0.2, stat = 'probability')

plt.xlabel('Map correlation (r)')
plt.show()

In [ ]:
bins_position = 30
fr_maps_all = []

idx_plot = 0


PCs_b_all_sal_PFC = []
PCs_d1_all_sal_PFC = []

for irec in range(0,len(x)):
    

    ksort_path = ksort_HPC_path_matched[irec]
    print('experiment',ksort_path)

    size_to = []
    size_to = np.min([spikes_binned_all_HPC[irec].shape[1],beh_binned_all[irec][0].size]) - 1



    # baseline
    rate_maps_temp_b= []
    rate_maps_temp_b= fr_map_ang(spikes_binned_all_PFC[irec][:,0:baseline][:,mask_mov_all[irec][0:baseline]],
                                                                      np.array([beh_binned_all[irec][0][0:baseline][mask_mov_all[irec][0:baseline]],
                                                                                beh_binned_all[irec][1][0:baseline][mask_mov_all[irec][0:baseline]]]),
                                                                                bins_position,10)

    # baseline
    rate_maps_temp_d1= []
    rate_maps_temp_d1= fr_map_ang(spikes_binned_all_PFC[irec][:,baseline:dose1][:,mask_mov_all[irec][baseline:dose1]],
                                                                      np.array([beh_binned_all[irec][0][baseline:dose1][mask_mov_all[irec][baseline:dose1]],
                                                                                beh_binned_all[irec][1][baseline:dose1][mask_mov_all[irec][baseline:dose1]]]),
                                                                                bins_position,10)    
    
    if (drug_list[irec] == 'SAL'):
    
        fig = plt.figure(figsize=(2,2))
        ax = fig.add_subplot(1,2,1)

        idx_sort = np.nanargmax(np.vstack(rate_maps_temp_b)[PC_test_b_all_PFC[irec],:].T,axis=0)
        ax.imshow(((np.vstack(rate_maps_temp_b)[PC_test_b_all_PFC[irec],:]).T[:,np.argsort(idx_sort)]).T)
        ax.set_title('Baseline')
        ax.set_xlabel('Angle')
        ax.set_ylabel('Cell ID')

        ax2 = fig.add_subplot(1,2,2)
        ax2.imshow(((np.vstack(rate_maps_temp_d1)[PC_test_b_all_PFC[irec],:]).T[:,np.argsort(idx_sort)]).T)    
        ax2.axis(False)
        
#         plt.savefig(r'G:\My Drive\Fenton_Lab\Ketamine_analysis\Manuscript\PC_linearized_PFC_SAL' + str(idx_plot)  + '.pdf', transparent = True)
#         plt.show()
#         plt.clf()
#         plt.close()  
        
        PCs_b_all_sal_PFC.append((np.vstack(rate_maps_temp_b)[PC_test_b_all_PFC[irec],:]).T)
        PCs_d1_all_sal_PFC.append((np.vstack(rate_maps_temp_d1)[PC_test_b_all_PFC[irec],:]).T)

        idx_plot = idx_plot +1

In [ ]:
print(np.hstack(PCs_b_all_sal).shape, np.hstack(PCs_b_all_sal_PFC).shape)

In [ ]:
## plotting all PCs during baseline and dose 1

fig = plt.figure(figsize=(10,10))
ax = fig.add_subplot(1,2,1)

idx_sort = np.nanargmax(np.hstack(PCs_b_all_sal_PFC),axis=0)

ax.imshow(np.hstack(PCs_b_all_sal_PFC)[:,np.argsort(idx_sort)].T)
ax.set_title('Saline')
ax.set_xlabel('Angle')
ax.set_ylabel('Cell ID')

ax2 = fig.add_subplot(1,2,2)
ax2.imshow(np.hstack(PCs_d1_all_sal_PFC)[:,np.argsort(idx_sort)].T)    
ax2.set_title('Saline')
ax2.axis(False)
# plt.savefig(r'G:\My Drive\Fenton_Lab\Ketamine_analysis\Manuscript\PC_linearized_PFC_SAL_summary' + '.pdf', transparent = True)
# plt.show()
# plt.clf()
# plt.close()  

## map correlation

map_corr_PFC = []
map_corr_PFC = [pearsonr(p,q).statistic for p,q in zip(np.hstack(PCs_b_all_sal_PFC).T,np.hstack(PCs_d1_all_sal_PFC).T)]

map_corr_PFC_shuf = []
for ishuf in range(0,100):
    PC_b_shuf = np.hstack(PCs_b_all_sal_PFC).T.copy()
    np.random.shuffle(PC_b_shuf)

    map_corr_PFC_shuf_tmp = []
    map_corr_PFC_shuf_tmp = [pearsonr(p,q).statistic for p,q in zip(np.hstack(PCs_b_all_sal_PFC).T,PC_b_shuf)]
    map_corr_PFC_shuf.append(np.hstack(map_corr_PFC_shuf_tmp))

print('mean map corr',np.nanmean(np.hstack(map_corr_PFC)))
print('mean map corr_shuf',np.nanmean(np.concatenate(map_corr_PFC_shuf,axis=None)))

noise = np.random.normal(0,0.5,np.hstack(PCs_b_all_sal_PFC).T.shape)
PCs_b_all_sal_PFC_fake = np.hstack(PCs_b_all_sal_PFC).T + noise

map_corr_PFC_fake = []
map_corr_PFC_fake = [pearsonr(p,q).statistic for p,q in zip(np.hstack(PCs_b_all_sal_PFC).T,PCs_b_all_sal_PFC_fake)]

print('mean map corr',np.nanmean(np.hstack(map_corr_PFC_fake)))


## histogram

bins = np.arange(-1,1,0.1)
fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot(1,1,1)
sns.histplot(np.hstack(map_corr_PFC), bins=bins, color ='orange', alpha= 0.3, stat = 'probability',fill = False)
sns.histplot(np.hstack(map_corr_HPC), bins=bins, color ='green', alpha= 0.3, stat = 'probability',fill = False)

sns.histplot(np.hstack(map_corr_PFC_shuf), bins=bins, color = 'black',alpha = 0.2, stat = 'probability', fill = False)
sns.histplot(np.hstack(map_corr_PFC_fake), bins=bins, color = 'red',alpha = 0.2, stat = 'probability',fill = False)

plt.xlabel('Map correlation (r)')


# plt.savefig(r'G:\My Drive\Fenton_Lab\Ketamine_analysis\Manuscript\map_corr_HPC_PFC' + '.pdf', transparent = True)
# plt.show()
# plt.clf()
# plt.close()  


In [ ]:
## histogram

bins = np.arange(-1,1,0.1)
fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot(1,1,1)
sns.histplot(np.hstack(map_corr_PFC), bins=bins, color ='orange', alpha= 0.3, stat = 'probability',fill = False)
# sns.histplot(np.hstack(map_corr_HPC), bins=bins, color ='green', alpha= 0.3, stat = 'probability',fill = False)

sns.histplot(np.hstack(map_corr_PFC_shuf), bins=bins, color = 'black',alpha = 0.2, stat = 'probability', fill = False)
# sns.histplot(np.hstack(map_corr_PFC_fake), bins=bins, color = 'red',alpha = 0.2, stat = 'probability',fill = False)

plt.xlabel('Map correlation (r)')

# plt.savefig(r'G:\My Drive\Fenton_Lab\Ketamine_analysis\Manuscript\map_corr_PFC' + '.pdf', transparent = True)
# plt.show()
# plt.clf()
# plt.close()  


In [ ]:
def fr_map(spk_data, beh_data, binsxy, fsrate):

    occ, rate_maps_temp= [], []

    # COMPUTING THE RATE MAPS
    occ = np.histogram2d(beh_data[0], beh_data[1], bins=binsxy)[0] / fsrate
    rate_maps_temp = [np.histogram2d(beh_data[0], beh_data[1], bins=binsxy, weights=s)[0]
                      / occ for s in spk_data]
    return rate_maps_temp

In [ ]:
def ratemap_comparison(fr_map_data, PC_data):
    from scipy.ndimage import gaussian_filter
    
    bas_fr, d1_fr, d2_fr, d3_fr = [], [], [], []
    
    idx_nan = [np.isnan(p) for p in fr_map_data[0]]
    bas_fr = [gaussian_filter(np.nan_to_num(p), sigma=1.5) for p in fr_map_data[0]]
    for icell in range(0,len(bas_fr)):
        bas_fr[icell][idx_nan[icell]] = np.nan    

    idx_nan = [np.isnan(p) for p in fr_map_data[1]]    
    d1_fr = [gaussian_filter(np.nan_to_num(p), sigma=1.5) for p in fr_map_data[1]]
    for icell in range(0,len(d1_fr)):
        d1_fr[icell][idx_nan[icell]] = np.nan       
    
    
    idx_nan = [np.isnan(p) for p in fr_map_data[2]]    
    d2_fr =[gaussian_filter(np.nan_to_num(p), sigma=1.5) for p in fr_map_data[2]]
    for icell in range(0,len(d2_fr)):
        d2_fr[icell][idx_nan[icell]] = np.nan      
        
    idx_nan = [np.isnan(p) for p in fr_map_data[3]]    
    d3_fr = [gaussian_filter(np.nan_to_num(p), sigma=1.5) for p in fr_map_data[3]]
    for icell in range(0,len(d3_fr)):
        d3_fr[icell][idx_nan[icell]] = np.nan        
    
    if np.sum(PC_data) > 2:
        for icell in np.hstack(np.argwhere(np.hstack(PC_data) == True)):
            print('Cell', str(icell))

            fig = plt.figure(figsize =(8,2))
            ax = fig.add_subplot(1,4,1)
            sns.heatmap(bas_fr[icell], cmap='jet', ax=ax, cbar_kws={'label': 'Normalized rate'}, cbar = False)
    #         ax.set_xlabel('X position')
    #         ax.set_ylabel('Y position')
            ax.axis(False)
            fname = 'Baseline'
            ax.set_title(fname)

            ax2 = fig.add_subplot(1,4,2)
            sns.heatmap(d1_fr[icell], cmap='jet', ax=ax2, cbar_kws={'label': 'Normalized rate'}, cbar = False)
    #         ax2.set_xlabel('X position')
    #         ax2.set_ylabel('Y position')
            ax2.axis(False)
            fname = 'Dose 1'
            ax2.set_title(fname)

            ax3 = fig.add_subplot(1,4,3)
            sns.heatmap(d2_fr[icell], cmap='jet', ax=ax3, cbar_kws={'label': 'Normalized rate'}, cbar = False)
    #         ax3.set_xlabel('X position')
    #         ax3.set_ylabel('Y position')
            ax3.axis(False)
            fname = 'Dose 2'
            ax3.set_title(fname)

            ax4 = fig.add_subplot(1,4,4)
            sns.heatmap(d3_fr[icell], cmap='jet', ax=ax4, cbar_kws={'label': 'Normalized rate'}, cbar = False)
    #         ax4.set_xlabel('X position')
    #         ax4.set_ylabel('Y position')
            ax4.axis(False)
            fname = 'Dose 3'
            ax4.set_title(fname)   

            plt.show()

    return

In [ ]:
bins_position = 15
fr_maps_all = []

for irec in range(0,1): #len(x)):
    
    if (drug_list[irec] == 'SAL'):

        ksort_path = ksort_HPC_path_matched[irec]
        print('experiment',ksort_path)

        size_to = []
        size_to = np.min([spikes_binned_all_HPC[irec].shape[1],beh_binned_all[irec][0].size]) - 1



        # baseline
        rate_maps_temp_b= []
        rate_maps_temp_b= fr_map(spikes_binned_all_HPC[irec][:,0:baseline][:,mask_mov_all[irec][0:baseline]],
                                                                          np.array([beh_binned_all[irec][0][0:baseline][mask_mov_all[irec][0:baseline]],
                                                                                    beh_binned_all[irec][1][0:baseline][mask_mov_all[irec][0:baseline]]]),
                                                                                    bins_position,10)


        # dose 1
        rate_maps_temp_d1 = []  
        rate_maps_temp_d1 = fr_map(spikes_binned_all_HPC[irec][:,baseline:dose1][:,mask_mov_all[irec][baseline:dose1]],
                                                                          np.array([beh_binned_all[irec][0][baseline:dose1][mask_mov_all[irec][baseline:dose1]],
                                                                                    beh_binned_all[irec][1][baseline:dose1][mask_mov_all[irec][baseline:dose1]]]),
                                                                                         bins_position,10)


        # dose 2
        rate_maps_temp_d2= [] 
        rate_maps_temp_d2= fr_map(spikes_binned_all_HPC[irec][:,dose1:dose2][:,mask_mov_all[irec][dose1:dose2]],
                                                                          np.array([beh_binned_all[irec][0][dose1:dose2][mask_mov_all[irec][dose1:dose2]],
                                                                                    beh_binned_all[irec][1][dose1:dose2][mask_mov_all[irec][dose1:dose2]]]),
                                                                                         bins_position,10)


        # dose 3
        rate_maps_temp_d3 = [] 
        rate_maps_temp_d3 = fr_map(spikes_binned_all_HPC[irec][:,dose2:size_to][:,mask_mov_all[irec][dose2:size_to]],
                                                                          np.array([beh_binned_all[irec][0][dose2:size_to][mask_mov_all[irec][dose2:size_to]],
                                                                                    beh_binned_all[irec][1][dose2:size_to][mask_mov_all[irec][dose2:size_to]]]),
                                                                                         bins_position,10)

        fr_maps_all.append([rate_maps_temp_b, rate_maps_temp_d1, rate_maps_temp_d2, rate_maps_temp_d3])

        ratemap_comparison([rate_maps_temp_b, rate_maps_temp_d1, rate_maps_temp_d2, rate_maps_temp_d3], PC_test_b_all[irec])

In [ ]:
bins_position = 15
ls_tmp = []
fr_maps_all = []

for irec in range(0,1): #len(x)):
    
    if (drug_list[irec] == 'SAL'):

        ksort_path = ksort_HPC_path_matched[irec]
        print('experiment',ksort_path)

        size_to = []
        size_to = np.min([spikes_binned_all_HPC[irec].shape[1],beh_binned_all[irec][0].size]) - 1
        
        
        
        # baseline
        rate_maps_temp_b= []
        rate_maps_temp_b= fr_map(spikes_binned_all_PFC[irec][:,0:baseline],
                                                                          np.array([beh_binned_all[irec][0][0:baseline],
                                                                                    beh_binned_all[irec][1][0:baseline]]),
                                                                                    bins_position,10)
        
        
        # dose 1
        rate_maps_temp_d1 = []  
        rate_maps_temp_d1 = fr_map(spikes_binned_all_PFC[irec][:,baseline:dose1],
                                                                          np.array([beh_binned_all[irec][0][baseline:dose1],
                                                                                    beh_binned_all[irec][1][baseline:dose1]]),
                                                                                         bins_position,10)
    
        
        # dose 2
        rate_maps_temp_d2= [] 
        rate_maps_temp_d2= fr_map(spikes_binned_all_PFC[irec][:,dose1:dose2],
                                                                          np.array([beh_binned_all[irec][0][dose1:dose2],
                                                                                    beh_binned_all[irec][1][dose1:dose2]]),
                                                                                         bins_position,10)
        
        
        # dose 3
        rate_maps_temp_d3 = [] 
        rate_maps_temp_d3 = fr_map(spikes_binned_all_PFC[irec][:,dose2:size_to],
                                                                          np.array([beh_binned_all[irec][0][dose2:size_to],
                                                                                    beh_binned_all[irec][1][dose2:size_to]]),
                                                                                         bins_position,10)
        
        fr_maps_all.append([rate_maps_temp_b, rate_maps_temp_d1, rate_maps_temp_d2, rate_maps_temp_d3])
        
        ratemap_comparison([rate_maps_temp_b, rate_maps_temp_d1, rate_maps_temp_d2, rate_maps_temp_d3],PC_test_b_all_PFC[irec])

### space coverage - return time - Markov process - occupancy and probability of being to the bin

In [ ]:
pip install discreteMarkovChain

In [ ]:
def compute_transition_matrix2(data, n, step = 1):
    
    t = np.array(data)
    step = step
    total_inds = t.size - (step + 1) + 1
    t_strided = np.lib.stride_tricks.as_strided(
                                    t,
                                    shape = (total_inds, 2),
                                    strides = (t.strides[0], step * t.strides[0]))
    
    inds, counts = np.unique(t_strided, axis = 0, return_counts = True)

    P = np.zeros((n, n))
    P[inds[:, 0], inds[:, 1]] = counts
    
    sums = P.sum(axis = 1)
    # Avoid divide by zero error by normalizing only non-zero rows
    P[sums != 0] = P[sums != 0] / sums[sums != 0][:, None]
    
    # P = P / P.sum(axis = 1)[:, None]
    return P

In [ ]:
## return probability

from typing import List
 
# Function to multiply two matrices A and B
def multiply(A: List[List[float]], B: List[List[float]],
             N: int) -> List[List[float]]:
    C = [[0 for _ in range(N)] for _ in range(N)]
    for i in range(N):
        for j in range(N):
            for k in range(N):
                C[i][j] += A[i][k] * B[k][j]
    return C
 
# Function to calculate the power of a matrix
def matrix_power(M: List[List[float]], p: int, n: int) -> List[List[float]]:
    A = [[0 for _ in range(n)] for _ in range(n)]
    for i in range(n):
        A[i][i] = 1
    while (p):
        if (p % 2):
            A = multiply(A, M, n)
        M = multiply(M, M, n)
        p //= 2
    return A
 
# Function to calculate the probability of
# reaching F at time T after starting from S
def findProbability(M: List[List[float]], N: int, F: int, S: int,
                    T: int) -> float:
 
    # Storing M^T in MT
    MT = matrix_power(M, T, N)
 
    # Returning the answer
    return MT[F - 1][S - 1]

In [ ]:
import scipy

transition_matrix_all = []

for irec in range(0,len(x_aln)):

    ksort_path = ksort_HPC_path_matched[irec]
    print('experiment',ksort_path)
    
    observations = []
    observations = scipy.stats.binned_statistic_2d(beh_binned_all[irec][0], beh_binned_all[irec][1],None,
                                    statistic='count', bins=4, range=[(-150,150),(-150,150)], expand_binnumbers=False)[3]

    transition_matrix = []
    transition_matrix = compute_transition_matrix2(observations, np.unique(observations).max()+1, 1)

    fig = plt.figure(figsize=(6,6))
    ax = fig.add_subplot(1,1,1)
    sns.heatmap(transition_matrix, vmax= 0.5,cmap = 'viridis', ax=ax, cbar_kws={'label': 'Transition probability'})
    ax.set_xlabel('State')
    ax.set_ylabel('State')
    plt.show()       
    
    transition_matrix_all.append(transition_matrix)

In [ ]:
## return probability
G = []
G = transition_matrix_all[10] # transition matrix of the Markov process

N = transition_matrix_all[10].shape[0] # number of different states
S = 20 # 1st state
F = 10 # 2nd state
T = 100 # in samples (in this case, seconds)

print(
    "The probability of reaching {} at time {}\nafter starting from {} is {}\n"
    .format(F, T, S, findProbability(G, N, F, S, T)))

## G is transition matrix
## N is number of states
## F where to go
## S from where
## T how many samples

# Cofiring analysis

In [ ]:
## concatenate activity vectors of HPC and PFC, this is done at the cell level, not time

spikes_binned_HPC_PFC = []
for irec in range(0,len(x_aln)):
    
    spikes_binned_HPC_PFC.append(np.vstack([spikes_binned_all_HPC[irec],spikes_binned_all_PFC[irec]]))
    print('HPC_cells',spikes_binned_all_HPC[irec].shape, 'PFC_cells',
          spikes_binned_all_PFC[irec].shape, 'total', spikes_binned_HPC_PFC[irec].shape)
    

In [ ]:
def pairwise_dist(depths_data):
    import itertools

    dist = np.concatenate([x1-x2 for x1, x2 in itertools.combinations(depths_data, 2)],axis=None)
    
    dist[(dist < -200)*(dist > 200)] = np.nan

    return dist

In [ ]:
def kendalltau_distance_mPFC(data1, data2):
    PCo_out = []
    
    data1 = data1[:, 0]

    idx = (np.isnan(data1))|(np.isnan(data2))
    data1 = data1[~idx]
    data2 = data2[~idx]
#     
    try:
        a, b = np.polyfit(data1, data2, 1)

    except:
        a = 0
        b = 0
        
    PCo_out = pearsonr(data1, data2)
# 
    fig = plt.figure(figsize=(5, 5))
    ax = fig.add_subplot(1,1,1)
    # Creating plot
    
#     sns.kdeplot(x=data1,y=data2, fill=True, ax = ax)
    ax.scatter(data1, data2, s=20, color='orange', alpha=0.3)
    plt.xlabel('Tau corr.')
    plt.ylabel('Distance (mm)')
    # plt.xlim(0,1)

    print('PCo', PCo_out.statistic)

    return PCo_out

In [ ]:
## Kendall tau correlations - all times - cell pair correlations

## do in HPC

PCo_sal_all_mov = []
tauVec_sal_all_mov = []

idx_plot = 0
for irec in range(0,1): #len(x_aln)):
    
    if (drug_list[irec] == 'SAL'):


        ksort_path = ksort_HPC_path_matched[irec]
        print('experiment',ksort_path)

        corr_baseline, cellpairs_baseline, tauVec_baseline = [],[],[]
        corr_dose1, cellpairs_dose1, tauVec_dose1 = [],[],[]

        corr_baseline, cellpairs_baseline, tauVec_baseline = kendall_tau_parallel(spikes_binned_all_PFC[irec][:,0:baseline][:,mask_mov_all[irec][0:baseline]])
#         corr_dose1, cellpairs_dose1, tauVec_dose1 = kendall_tau_parallel(spikes_binned_all_PFC[irec][:,baseline:dose1][:,mask_mov_all[irec][baseline:dose1]])

        dist_tmp = pairwise_dist(depth_PFC_all[irec])
        
        print('Baseline')
        PCo_PFC = kendalltau_distance_mPFC(tauVec_baseline[0],dist_tmp)  
        plt.ylim(-100,100)
        plt.show()
#         plt.savefig(r'G:\My Drive\Fenton_Lab\Ketamine_analysis\Manuscript\PCo_PFC' + str(idx_plot) + '.pdf', transparent = True)
#         plt.show()
#         plt.clf()
#         plt.close()  
        
#         print('Dose 1')
#         PCo_HPC_PFC = kendalltau_distance_mPFC(tauVec_dose1[0], dist_tmp)    
#         plt.show()
#         plt.savefig(r'G:\My Drive\Fenton_Lab\Ketamine_analysis\Manuscript\PCo_HPC_PFC' + str(idx_plot) + '.pdf', transparent = True)
#         plt.show()
#         plt.clf()
#         plt.close()  

        idx_plot = idx_plot +1

In [ ]:
def kendall_visualization_PCo_doses(data1, data2, data3, data4):
    PCo_out = []
    
    data1 = data1[:, 0]
    data2 = data2[:, 0]
    data3 = data3[:, 0]
    data4 = data4[:, 0]
    
    idx = (np.isnan(data1))|(np.isnan(data2))|(np.isnan(data3))|(np.isnan(data4))
    data1 = data1[~idx]
    data2 = data2[~idx]    
    data3 = data3[~idx]
    data4 = data4[~idx] 
    
    try:
        a, b = np.polyfit(data1, data2, 1)

    except:
        a = 0
        b = 0

    try:
        a2, b2 = np.polyfit(data1, data3, 1)

    except:
        a2 = 0
        b2 = 0

    try:
        a3, b3 = np.polyfit(data1, data4, 1)

    except:
        a3 = 0
        b3 = 0

    if data1.size < 2: 
        print('not enough data')
        PCo_out=[np.nan, np.nan, np.nan]
        return PCo_out
    
    PCo_out = [pearsonr(data1, data2),
               pearsonr(data1, data3),
               pearsonr(data1, data4)]

    fig = plt.figure(figsize=(5, 5))
    # Creating plot
    plt.scatter(data1, data2, s=20, color='green', alpha=0.3)
    plt.scatter(data1, data3, s=20, color='orange', alpha=0.3)
    plt.scatter(data1, data4, s=20, color='blue', alpha=0.3)

    x = np.linspace(-1, 1, 10)
    plt.plot(x, x, '--k')
    plt.plot(x, a * x + b, linewidth=6, color='green', alpha=0.5, label='Dose 1')
    plt.plot(x, a2 * x + b2, linewidth=6, color='orange', alpha=0.5, label='Dose 2')
    plt.plot(x, a3 * x + b3, linewidth=6, color='blue', alpha=0.5, label='Dose 3')

    plt.xlabel('Pairwise correlations - Baseline')
    plt.ylabel('Pairwise correlations')
    plt.legend()
    # plt.xlim(0,1)
#     plt.show()

    print('PCo', PCo_out[0].statistic, PCo_out[1].statistic, PCo_out[2].statistic)

    return PCo_out

In [ ]:
def kendall_visualization_PCo(data1, data2):
    PCo_out = []
    
    data1 = data1[:, 0]
    data2 = data2[:, 0]

    idx = (np.isnan(data1))|(np.isnan(data2))
    data1 = data1[~idx]
    data2 = data2[~idx]
#     
    try:
        a, b = np.polyfit(data1, data2, 1)

    except:
        a = 0
        b = 0
    if data1.size < 2: 
        print('not enough data')
        PCo_out = np.nan
        return PCo_out
    
    PCo_out = pearsonr(data1, data2)
# 
    fig = plt.figure(figsize=(5, 5))
    # Creating plot
    plt.scatter(data1, data2, s=50, color='orange', alpha=0.3)
    x = np.linspace(-1, 1, 10)
    plt.plot(x, x, '--k')
    plt.plot(x, a * x + b, linewidth=6, color='orange', alpha=0.5)
    plt.xlabel('Pairwise correlations - Condition 1')
    plt.ylabel('Pairwise correlations - Condition 2')
    # plt.xlim(0,1)

    print('PCo', PCo_out.statistic)

    return PCo_out

In [ ]:
## Kendall tau correlations - all times - cell pair correlations
PCo_sal_all_mov = []
tauVec_sal_all_mov = []

idx_plot = 0
for irec in range(0,len(x_aln)):
    
    if (drug_list[irec] == 'SAL'):

        size_to = np.min([dose3, mask_mov_all[irec].size])

        ksort_path = ksort_HPC_path_matched[irec]
        print('experiment',ksort_path)

        corr_baseline, cellpairs_baseline, tauVec_baseline = [],[],[]
        corr_dose1, cellpairs_dose1, tauVec_dose1 = [],[],[]

        corr_baseline, cellpairs_baseline, tauVec_baseline = kendall_tau_parallel(spikes_binned_HPC_PFC[irec][:,0:baseline][:,mask_mov_all[irec][0:baseline]])
        corr_dose1, cellpairs_dose1, tauVec_dose1 = kendall_tau_parallel(spikes_binned_HPC_PFC[irec][:,baseline:dose1][:,mask_mov_all[irec][baseline:dose1]])
        corr_dose2, cellpairs_dose2, tauVec_dose2 = kendall_tau_parallel(spikes_binned_HPC_PFC[irec][:,dose1:dose2][:,mask_mov_all[irec][dose1:dose2]])
        corr_dose3, cellpairs_dose3, tauVec_dose3 = kendall_tau_parallel(spikes_binned_HPC_PFC[irec][:,dose2:size_to][:,mask_mov_all[irec][dose2:size_to]])

        ## mask for pairs of cells from HPC-HPC / PFC-PFC / HPC-PFC
        HPC_idx = np.arange(0,spikes_binned_all_HPC[irec].shape[0])  #Change index depending of the recording - change the variable names

        test = []
        test = [np.intersect1d(HPC_idx,np.hstack(p)).size for p in cellpairs_baseline]

        mask_HPC_HPC = np.zeros(len(cellpairs_baseline)).astype(bool)
        mask_HPC_HPC[np.hstack(test)==2] = True

        mask_HPC_PFC = np.zeros(len(cellpairs_baseline)).astype(bool)
        mask_HPC_PFC[np.hstack(test)==1] = True

        mask_PFC_PFC = np.zeros(len(cellpairs_baseline)).astype(bool)
        mask_PFC_PFC[np.hstack(test)==0] = True

        # PCo computation
        PCo_HPC,PCo_PFC,PCo_HPC_PFC = [],[],[]

        print('Hippocampal pair of cells')
        PCo_HPC = kendall_visualization_PCo(tauVec_baseline[0][mask_HPC_HPC],tauVec_dose1[0][mask_HPC_HPC])
        PCo_HPC2 = kendall_visualization_PCo(tauVec_dose1[0][mask_HPC_HPC],tauVec_dose2[0][mask_HPC_HPC])
        PCo_HPC3 = kendall_visualization_PCo(tauVec_dose2[0][mask_HPC_HPC],tauVec_dose3[0][mask_HPC_HPC])
        
#         plt.savefig(r'G:\My Drive\Fenton_Lab\Ketamine_analysis\Manuscript\PCo_HPC' + str(idx_plot) + '.pdf', transparent = True)
        plt.show()
#         plt.clf()
#         plt.close()  
        
        print('PFC pair of cells')
        PCo_PFC = kendall_visualization_PCo(tauVec_baseline[0][mask_PFC_PFC],tauVec_dose1[0][mask_PFC_PFC])  
        PCo_PFC2 = kendall_visualization_PCo(tauVec_dose1[0][mask_PFC_PFC],tauVec_dose2[0][mask_PFC_PFC])  
        PCo_PFC3 = kendall_visualization_PCo(tauVec_dose2[0][mask_PFC_PFC],tauVec_dose3[0][mask_PFC_PFC])  
        
        
#         plt.savefig(r'G:\My Drive\Fenton_Lab\Ketamine_analysis\Manuscript\PCo_PFC' + str(idx_plot) + '.pdf', transparent = True)
        plt.show()
#         plt.clf()
#         plt.close()  
        
        print('HPC-PFC pair of cells')
        PCo_HPC_PFC = kendall_visualization_PCo(tauVec_baseline[0][mask_HPC_PFC],tauVec_dose1[0][mask_HPC_PFC])         
        PCo_HPC_PFC2 = kendall_visualization_PCo(tauVec_dose1[0][mask_HPC_PFC],tauVec_dose2[0][mask_HPC_PFC])         
        PCo_HPC_PFC3 = kendall_visualization_PCo(tauVec_dose2[0][mask_HPC_PFC],tauVec_dose3[0][mask_HPC_PFC])         
        
#         plt.savefig(r'G:\My Drive\Fenton_Lab\Ketamine_analysis\Manuscript\PCo_HPC_PFC' + str(idx_plot) + '.pdf', transparent = True)
        plt.show()
#         plt.clf()
#         plt.close()  
        
        PCo_sal_all_mov.append([[PCo_HPC,PCo_HPC2, PCo_HPC3],
                               [PCo_PFC,PCo_PFC2,PCo_PFC3],
                               [PCo_HPC_PFC,PCo_HPC_PFC2,PCo_HPC_PFC3]])

        ## save tauVec, cell pairs

        tauVec_sal_all_mov.append([tauVec_baseline,tauVec_dose1,tauVec_dose2,tauVec_dose3])
        idx_plot = idx_plot +1

In [ ]:
np.concatenate(PCo_sal_all_mov,axis=1)[0,:,0]

In [ ]:
df_sal_PCo = []
df_sal_PCo = pd.concat([pd.DataFrame({'PCo':np.concatenate(PCo_sal_all_mov,axis=1)[0,:,0], 'brain_region': 'HPC-HPC'}),
                        pd.DataFrame({'PCo':np.concatenate(PCo_sal_all_mov,axis=1)[1,:,0], 'brain_region': 'PFC-PFC'}),
                        pd.DataFrame({'PCo':np.concatenate(PCo_sal_all_mov,axis=1)[2,:,0], 'brain_region': 'HPC-PFC'})])


df_sal_PCo.head(4)

In [ ]:
## PCo saline

fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot(1,1,1)
sns.boxplot(data=df_sal_PCo, y = 'PCo',x='brain_region', ax= ax)
sns.stripplot(data=df_sal_PCo, y = 'PCo',x='brain_region', dodge = True, color = 'black',ax=ax)

plt.savefig(r'G:\My Drive\Fenton_Lab\Ketamine_analysis\Manuscript\PCo_SAL' + '.pdf', transparent = True)
plt.show()
plt.clf()
plt.close()  

In [ ]:
## Kendall tau correlations - all times - cell pair correlations
PCo_sal_all_mov = []
tauVec_sal_all_mov = []

idx_plot = 0
for irec in range(0,len(x_aln)):
    
    if (drug_list[irec] == 'SAL'):


        ksort_path = ksort_HPC_path_matched[irec]
        print('experiment',ksort_path)

        corr_baseline, cellpairs_baseline, tauVec_baseline = [],[],[]
        corr_dose1, cellpairs_dose1, tauVec_dose1 = [],[],[]

        bas_shuf = spikes_binned_HPC_PFC[irec][:,0:baseline][:,mask_mov_all[irec][0:baseline]].copy()
        dos1_shuf = spikes_binned_HPC_PFC[irec][:,baseline:dose1][:,mask_mov_all[irec][baseline:dose1]].copy()
        np.random.shuffle(bas_shuf)
        np.random.shuffle(dos1_shuf)
        
        corr_baseline, cellpairs_baseline, tauVec_baseline = kendall_tau_parallel(bas_shuf)
        corr_dose1, cellpairs_dose1, tauVec_dose1 = kendall_tau_parallel(dos1_shuf)

        ## mask for pairs of cells from HPC-HPC / PFC-PFC / HPC-PFC
        HPC_idx = np.arange(0,spikes_binned_all_HPC[irec].shape[0])  #Change index depending of the recording - change the variable names

        test = []
        test = [np.intersect1d(HPC_idx,np.hstack(p)).size for p in cellpairs_baseline]

        mask_HPC_HPC = np.zeros(len(cellpairs_baseline)).astype(bool)
        mask_HPC_HPC[np.hstack(test)==2] = True

        mask_HPC_PFC = np.zeros(len(cellpairs_baseline)).astype(bool)
        mask_HPC_PFC[np.hstack(test)==1] = True

        mask_PFC_PFC = np.zeros(len(cellpairs_baseline)).astype(bool)
        mask_PFC_PFC[np.hstack(test)==0] = True

        # PCo computation
        PCo_HPC,PCo_PFC,PCo_HPC_PFC = [],[],[]

        print('Hippocampal pair of cells')
        PCo_HPC = kendall_visualization_PCo(tauVec_baseline[0][mask_HPC_HPC],tauVec_dose1[0][mask_HPC_HPC])
        plt.show()
        
        print('PFC pair of cells')
        PCo_PFC = kendall_visualization_PCo(tauVec_baseline[0][mask_PFC_PFC],tauVec_dose1[0][mask_PFC_PFC])        
        plt.show()
        
        print('HPC-PFC pair of cells')
        PCo_HPC_PFC = kendall_visualization_PCo(tauVec_baseline[0][mask_HPC_PFC],tauVec_dose1[0][mask_HPC_PFC])         
        plt.show()


In [ ]:
## Kendall tau correlations - all times - cell pair correlations
PCo_all_mov = []
tauVec_all_mov = []

drug_reduced_list = []
mouse_reduced_list = []

for irec in range(0,len(x_aln)):

    if (drug_list[irec] == 'SAL') or (drug_list[irec] == 'RSK'):

        ksort_path = ksort_HPC_path_matched[irec]
        print('experiment',ksort_path)
        size_to = np.min([dose3, mask_mov_all[irec].size])

        corr_baseline, cellpairs_baseline, tauVec_baseline = [],[],[]
        corr_dose1, cellpairs_dose1, tauVec_dose1 = [],[],[]
        corr_dose2, cellpairs_dose2, tauVec_dose2 = [],[],[]
        corr_dose3, cellpairs_dose3, tauVec_dose3 = [],[],[]

        corr_baseline, cellpairs_baseline, tauVec_baseline = kendall_tau_parallel(spikes_binned_HPC_PFC[irec][:,0:baseline][:,mask_mov_all[irec][0:baseline]])
        corr_dose1, cellpairs_dose1, tauVec_dose1 = kendall_tau_parallel(spikes_binned_HPC_PFC[irec][:,baseline:dose1][:,mask_mov_all[irec][baseline:dose1]])
        corr_dose2, cellpairs_dose2, tauVec_dose2 = kendall_tau_parallel(spikes_binned_HPC_PFC[irec][:,dose1:dose2][:,mask_mov_all[irec][dose1:dose2]])
        corr_dose3, cellpairs_dose3, tauVec_dose3 = kendall_tau_parallel(spikes_binned_HPC_PFC[irec][:,dose2:size_to][:,mask_mov_all[irec][dose2:size_to]])

        ## mask for pairs of cells from HPC-HPC / PFC-PFC / HPC-PFC
        HPC_idx = np.arange(0,spikes_binned_all_HPC[irec].shape[0])  #Change index depending of the recording - change the variable names

        test = []
        test = [np.intersect1d(HPC_idx,np.hstack(p)).size for p in cellpairs_baseline]

        mask_HPC_HPC = np.zeros(len(cellpairs_baseline)).astype(bool)
        mask_HPC_HPC[np.hstack(test)==2] = True

        mask_HPC_PFC = np.zeros(len(cellpairs_baseline)).astype(bool)
        mask_HPC_PFC[np.hstack(test)==1] = True

        mask_PFC_PFC = np.zeros(len(cellpairs_baseline)).astype(bool)
        mask_PFC_PFC[np.hstack(test)==0] = True

        # PCo computation
        PCo_HPC,PCo_PFC,PCo_HPC_PFC = [],[],[]

        print('Hippocampal pair of cells')
        PCo_HPC = kendall_visualization_PCo_doses(tauVec_baseline[0][mask_HPC_HPC],tauVec_dose1[0][mask_HPC_HPC],
                                                 tauVec_dose2[0][mask_HPC_HPC],tauVec_dose3[0][mask_HPC_HPC])
        plt.show()
        
        print('mPFC pair of cells')
        PCo_PFC = kendall_visualization_PCo_doses(tauVec_baseline[0][mask_PFC_PFC],tauVec_dose1[0][mask_PFC_PFC],
                                                 tauVec_dose2[0][mask_PFC_PFC],tauVec_dose3[0][mask_PFC_PFC])
        plt.show()

        print('mPFC-HPC pair of cells')
        PCo_HPC_PFC = kendall_visualization_PCo_doses(tauVec_baseline[0][mask_HPC_PFC],tauVec_dose1[0][mask_HPC_PFC],
                                                 tauVec_dose2[0][mask_HPC_PFC],tauVec_dose3[0][mask_HPC_PFC])
        
        plt.show()
        

        PCo_all_mov.append([PCo_HPC,PCo_PFC,PCo_HPC_PFC])
        drug_reduced_list.append(drug_list[irec])
        mouse_reduced_list.append(mouse_list[irec])
        ## save tauVec, cell pairs

        tauVec_all_mov.append([tauVec_baseline,tauVec_dose1,tauVec_dose2,tauVec_dose3])


In [ ]:

df_sal_rsk_PCo = []
ls_tmp = []
## Kendall tau correlations - all times - cell pair correlations

for it in range(0,len(mouse_reduced_list)):
    
    if np.sum(np.isnan(np.array(PCo_all_mov[it][0]))) == 3:
        continue

    ### create dataframe saving PCo values
    ls_tmp.append(pd.DataFrame({'drug':drug_reduced_list[it], 'condition': ['dose1', 'dose2', 'dose3'],
                                'PCo':np.array(PCo_all_mov[it][0])[:,0],'brain_region': 'HPC-HPC',
                                'mouse': mouse_reduced_list[it]}))

    ls_tmp.append(pd.DataFrame({'drug':drug_reduced_list[it], 'condition': ['dose1', 'dose2', 'dose3'],
                                'PCo':np.array(PCo_all_mov[it][1])[:,0],'brain_region': 'PFC-PFC',
                                'mouse': mouse_reduced_list[it]}))

    ls_tmp.append(pd.DataFrame({'drug':drug_reduced_list[it], 'condition': ['dose1', 'dose2', 'dose3'],
                                'PCo':np.array(PCo_all_mov[it][2])[:,0],'brain_region': 'HPC-PFC',
                                'mouse': mouse_reduced_list[it]}))    

In [ ]:
it

In [ ]:
df_sal_rsk_PCo = pd.concat(ls_tmp)
df_sal_rsk_PCo.head(4)

In [ ]:
## PCo saline rsk

fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot(1,1,1)

sns.boxplot(data=df_sal_rsk_PCo[df_sal_rsk_PCo['drug']=='SAL'],
            y = 'PCo',x='condition', hue = 'brain_region', ax= ax)

sns.stripplot(data=df_sal_rsk_PCo[df_sal_rsk_PCo['drug']=='SAL'],
              y = 'PCo',x='condition',hue = 'brain_region', dodge = True, color = 'black',ax=ax)

plt.show()
# plt.savefig(r'G:\My Drive\Fenton_Lab\Ketamine_analysis\Manuscript\PCo_SAL' + '.pdf', transparent = True)
# plt.show()
# plt.clf()
# plt.close()  

In [ ]:
## PCo saline rsk

fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot(1,1,1)

sns.boxplot(data=df_sal_rsk_PCo[df_sal_rsk_PCo['drug']=='RSK'],
            y = 'PCo',x='condition', hue = 'brain_region', ax= ax)

sns.stripplot(data=df_sal_rsk_PCo[df_sal_rsk_PCo['drug']=='RSK'],
              y = 'PCo',x='condition',hue = 'brain_region', dodge = True, color = 'black',ax=ax)

plt.show()
# plt.savefig(r'G:\My Drive\Fenton_Lab\Ketamine_analysis\Manuscript\PCo_SAL' + '.pdf', transparent = True)
# plt.show()
# plt.clf()
# plt.close()  

In [ ]:
## to modify lineplots

x1 = df_sal_rsk_PCo[(df_sal_rsk_PCo['drug'] == 'SAL') & (df_sal_rsk_PCo['condition'] == 'dose1')
                    & (df_sal_rsk_PCo['brain_region'] == 'HPC-HPC')].PCo

x2 = df_sal_rsk_PCo[(df_sal_rsk_PCo['drug'] == 'SAL') & (df_sal_rsk_PCo['condition'] == 'dose2')
                    & (df_sal_rsk_PCo['brain_region'] == 'HPC-HPC')].PCo

x3 = df_sal_rsk_PCo[(df_sal_rsk_PCo['drug'] == 'SAL') & (df_sal_rsk_PCo['condition'] == 'dose3')
                    & (df_sal_rsk_PCo['brain_region'] == 'HPC-HPC')].PCo

xtotal = np.array([x1, x2, x3])


x1 = df_sal_rsk_PCo[(df_sal_rsk_PCo['drug'] == 'RSK') & (df_sal_rsk_PCo['condition'] == 'dose1')
                    & (df_sal_rsk_PCo['brain_region'] == 'HPC-HPC')].PCo

x2 = df_sal_rsk_PCo[(df_sal_rsk_PCo['drug'] == 'RSK') & (df_sal_rsk_PCo['condition'] == 'dose2')
                    & (df_sal_rsk_PCo['brain_region'] == 'HPC-HPC')].PCo

x3 = df_sal_rsk_PCo[(df_sal_rsk_PCo['drug'] == 'RSK') & (df_sal_rsk_PCo['condition'] == 'dose3')
                    & (df_sal_rsk_PCo['brain_region'] == 'HPC-HPC')].PCo

xtotal2 = np.array([x1, x2, x3])

fig = plt.figure(figsize = (12,6))
ax = fig.add_subplot(1, 2, 1)

ax.plot(np.array([0, 1, 2]), xtotal, '-o',color = 'gray', alpha = 0.6, markeredgecolor = 'black', markeredgewidth = 3)
ax.plot(np.array([0, 1, 2]), xtotal2, '-o',color = 'green', alpha = 0.6, markeredgecolor = 'black', markeredgewidth = 3)

# sns.violinplot(data=df_sal_rsk_PCo[(df_sal_rsk_PCo['drug'] == 'SAL') & (df_sal_rsk_PCo['brain_region'] == 'HPC-HPC')],
#                x="condition", y="PCo", inner = None,showfliers = False, ax = ax)

# sns.stripplot(y="n_modes", hue = 'light',
#                 x="condition",dodge=True,
#                 data=df_registrations.reset_index(),
#               color="black", edgecolor="gray", ax = ax)

ax.legend([],[], frameon=False)
ax.set_ylabel('PCo')
ax.set_xticks([0,1,2])
ax.set_xticklabels(['Dose1', 'Dose2', 'Dose 3'])
# ax.set_ylim(-0.4,0.7)
# plt.show()
plt.savefig(r'G:\My Drive\Fenton_Lab\Ketamine_analysis\Manuscript\PCo_SAL_RSK_HPC' + '.pdf', transparent = True)
plt.show()
plt.clf()
plt.close()  


In [ ]:
## to modify lineplots

x1 = df_sal_rsk_PCo[(df_sal_rsk_PCo['drug'] == 'SAL') & (df_sal_rsk_PCo['condition'] == 'dose1')
                    & (df_sal_rsk_PCo['brain_region'] == 'PFC-PFC')].PCo

x2 = df_sal_rsk_PCo[(df_sal_rsk_PCo['drug'] == 'SAL') & (df_sal_rsk_PCo['condition'] == 'dose2')
                    & (df_sal_rsk_PCo['brain_region'] == 'PFC-PFC')].PCo

x3 = df_sal_rsk_PCo[(df_sal_rsk_PCo['drug'] == 'SAL') & (df_sal_rsk_PCo['condition'] == 'dose3')
                    & (df_sal_rsk_PCo['brain_region'] == 'PFC-PFC')].PCo

xtotal = np.array([x1, x2, x3])


x1 = df_sal_rsk_PCo[(df_sal_rsk_PCo['drug'] == 'RSK') & (df_sal_rsk_PCo['condition'] == 'dose1')
                    & (df_sal_rsk_PCo['brain_region'] == 'PFC-PFC')].PCo

x2 = df_sal_rsk_PCo[(df_sal_rsk_PCo['drug'] == 'RSK') & (df_sal_rsk_PCo['condition'] == 'dose2')
                    & (df_sal_rsk_PCo['brain_region'] == 'PFC-PFC')].PCo

x3 = df_sal_rsk_PCo[(df_sal_rsk_PCo['drug'] == 'RSK') & (df_sal_rsk_PCo['condition'] == 'dose3')
                    & (df_sal_rsk_PCo['brain_region'] == 'PFC-PFC')].PCo

xtotal2 = np.array([x1, x2, x3])

fig = plt.figure(figsize = (12,6))
ax = fig.add_subplot(1, 2, 1)

ax.plot(np.array([0, 1, 2]), xtotal, '-o',color = 'gray', alpha = 0.6, markeredgecolor = 'black', markeredgewidth = 3)
ax.plot(np.array([0, 1, 2]), xtotal2, '-o',color = 'orange', alpha = 0.6, markeredgecolor = 'black', markeredgewidth = 3)

# sns.violinplot(data=df_sal_rsk_PCo[(df_sal_rsk_PCo['drug'] == 'SAL') & (df_sal_rsk_PCo['brain_region'] == 'HPC-HPC')],
#                x="condition", y="PCo", inner = None,showfliers = False, ax = ax)

# sns.stripplot(y="n_modes", hue = 'light',
#                 x="condition",dodge=True,
#                 data=df_registrations.reset_index(),
#               color="black", edgecolor="gray", ax = ax)

ax.legend([],[], frameon=False)
ax.set_ylabel('PCo')
ax.set_xticks([0,1,2])
ax.set_xticklabels(['Dose1', 'Dose2', 'Dose 3'])
# ax.set_ylim(-0.4,0.7)
# plt.show()
plt.savefig(r'G:\My Drive\Fenton_Lab\Ketamine_analysis\Manuscript\PCo_SAL_RSK_PFC' + '.pdf', transparent = True)
plt.show()
plt.clf()
plt.close()  

In [ ]:
## to modify lineplots

x1 = df_sal_rsk_PCo[(df_sal_rsk_PCo['drug'] == 'SAL') & (df_sal_rsk_PCo['condition'] == 'dose1')
                    & (df_sal_rsk_PCo['brain_region'] == 'HPC-PFC')].PCo

x2 = df_sal_rsk_PCo[(df_sal_rsk_PCo['drug'] == 'SAL') & (df_sal_rsk_PCo['condition'] == 'dose2')
                    & (df_sal_rsk_PCo['brain_region'] == 'HPC-PFC')].PCo

x3 = df_sal_rsk_PCo[(df_sal_rsk_PCo['drug'] == 'SAL') & (df_sal_rsk_PCo['condition'] == 'dose3')
                    & (df_sal_rsk_PCo['brain_region'] == 'HPC-PFC')].PCo

xtotal = np.array([x1, x2, x3])


x1 = df_sal_rsk_PCo[(df_sal_rsk_PCo['drug'] == 'RSK') & (df_sal_rsk_PCo['condition'] == 'dose1')
                    & (df_sal_rsk_PCo['brain_region'] == 'HPC-PFC')].PCo

x2 = df_sal_rsk_PCo[(df_sal_rsk_PCo['drug'] == 'RSK') & (df_sal_rsk_PCo['condition'] == 'dose2')
                    & (df_sal_rsk_PCo['brain_region'] == 'HPC-PFC')].PCo

x3 = df_sal_rsk_PCo[(df_sal_rsk_PCo['drug'] == 'RSK') & (df_sal_rsk_PCo['condition'] == 'dose3')
                    & (df_sal_rsk_PCo['brain_region'] == 'HPC-PFC')].PCo

xtotal2 = np.array([x1, x2, x3])

fig = plt.figure(figsize = (12,6))
ax = fig.add_subplot(1, 2, 1)

ax.plot(np.array([0, 1, 2]), xtotal, '-o',color = 'gray', alpha = 0.6, markeredgecolor = 'black', markeredgewidth = 3)
ax.plot(np.array([0, 1, 2]), xtotal2, '-o',color = 'brown', alpha = 0.6, markeredgecolor = 'black', markeredgewidth = 3)

# sns.violinplot(data=df_sal_rsk_PCo[(df_sal_rsk_PCo['drug'] == 'SAL') & (df_sal_rsk_PCo['brain_region'] == 'HPC-HPC')],
#                x="condition", y="PCo", inner = None,showfliers = False, ax = ax)

# sns.stripplot(y="n_modes", hue = 'light',
#                 x="condition",dodge=True,
#                 data=df_registrations.reset_index(),
#               color="black", edgecolor="gray", ax = ax)

ax.legend([],[], frameon=False)
ax.set_ylabel('PCo')
ax.set_xticks([0,1,2])
ax.set_xticklabels(['Dose1', 'Dose2', 'Dose 3'])
# plt.show()
plt.savefig(r'G:\My Drive\Fenton_Lab\Ketamine_analysis\Manuscript\PCo_SAL_RSK_HPC_PFC' + '.pdf', transparent = True)
plt.show()
plt.clf()
plt.close()  

In [ ]:
df_sal_rsk_PCo_normchange = []
ls_tmp = []
## Kendall tau correlations - all times - cell pair correlations

for it in range(0,len(mouse_reduced_list)):
    
    if np.sum(np.isnan(np.array(PCo_all_mov[it][0]))) == 3:
        continue

#     ### create dataframe saving PCo values
#     ls_tmp.append(pd.DataFrame({'drug':drug_reduced_list[it], 'condition': ['dose2/1', 'dose3/2'],
#                                 'PCo':np.array(PCo_all_mov[it][0])[:,0][1:3]/np.array(PCo_all_mov[it][0])[:,0][0:2],'brain_region': 'HPC-HPC',
#                                 'mouse': mouse_reduced_list[it]}))

#     ls_tmp.append(pd.DataFrame({'drug':drug_reduced_list[it], 'condition': ['dose2/1', 'dose3/2'],
#                                 'PCo':np.array(PCo_all_mov[it][1])[:,0][1:3]/np.array(PCo_all_mov[it][1])[:,0][0:2],'brain_region': 'PFC-PFC',
#                                 'mouse': mouse_reduced_list[it]}))

#     ls_tmp.append(pd.DataFrame({'drug':drug_reduced_list[it], 'condition': ['dose2/1', 'dose3/2'],
#                                 'PCo':np.array(PCo_all_mov[it][2])[:,0][1:3]/np.array(PCo_all_mov[it][2])[:,0][0:2],'brain_region': 'HPC-PFC',
#                                 'mouse': mouse_reduced_list[it]}))    
  

    ### create dataframe saving PCo values
    ls_tmp.append(pd.DataFrame({'drug':drug_reduced_list[it], 'condition': ['dose2/1', 'dose3/2'],
                                'PCo':np.diff(np.array(PCo_all_mov[it][0])[:,0]),'brain_region': 'HPC-HPC',
                                'mouse': mouse_reduced_list[it]}))

    ls_tmp.append(pd.DataFrame({'drug':drug_reduced_list[it], 'condition': ['dose2/1', 'dose3/2'],
                                'PCo':np.diff(np.array(PCo_all_mov[it][1])[:,0]),'brain_region': 'PFC-PFC',
                                'mouse': mouse_reduced_list[it]}))

    ls_tmp.append(pd.DataFrame({'drug':drug_reduced_list[it], 'condition': ['dose2/1', 'dose3/2'],
                                'PCo':np.diff(np.array(PCo_all_mov[it][2])[:,0]),'brain_region': 'HPC-PFC',
                                'mouse': mouse_reduced_list[it]}))    

In [ ]:
df_sal_rsk_PCo_normchange = pd.concat(ls_tmp)
df_sal_rsk_PCo_normchange.head(4)

In [ ]:
## PCo saline rsk

fig = plt.figure(figsize=(3,3))
ax = fig.add_subplot(1,1,1)

sns.barplot(data=df_sal_rsk_PCo_normchange[df_sal_rsk_PCo_normchange['brain_region']=='HPC-HPC'],
            y = 'PCo',x='condition', hue = 'drug', ax= ax, errorbar = 'se')

sns.stripplot(data=df_sal_rsk_PCo_normchange[df_sal_rsk_PCo_normchange['brain_region']=='HPC-HPC'],
            y = 'PCo',x='condition', hue = 'drug', ax= ax, dodge = True, color = 'black')
ax.legend([],[], frameon=False)

plt.savefig(r'G:\My Drive\Fenton_Lab\Ketamine_analysis\Manuscript\PCo_barplot_HPC' + '.pdf', transparent = True)
plt.show()
plt.clf()
plt.close()  

In [ ]:
## PCo saline rsk

fig = plt.figure(figsize=(3,3))
ax = fig.add_subplot(1,1,1)

sns.barplot(data=df_sal_rsk_PCo_normchange[df_sal_rsk_PCo_normchange['brain_region']=='PFC-PFC'],
            y = 'PCo',x='condition', hue = 'drug', ax= ax, errorbar = 'se')

sns.stripplot(data=df_sal_rsk_PCo_normchange[df_sal_rsk_PCo_normchange['brain_region']=='PFC-PFC'],
            y = 'PCo',x='condition', hue = 'drug', ax= ax, dodge = True, color = 'black')
ax.legend([],[], frameon=False)

plt.savefig(r'G:\My Drive\Fenton_Lab\Ketamine_analysis\Manuscript\PCo_barplot_PFC' + '.pdf', transparent = True)
plt.show()
plt.clf()
plt.close()  

In [ ]:
## PCo saline rsk

fig = plt.figure(figsize=(3,3))
ax = fig.add_subplot(1,1,1)

sns.barplot(data=df_sal_rsk_PCo_normchange[df_sal_rsk_PCo_normchange['brain_region']=='HPC-PFC'],
            y = 'PCo',x='condition', hue = 'drug', ax= ax, errorbar = 'se')

sns.stripplot(data=df_sal_rsk_PCo_normchange[df_sal_rsk_PCo_normchange['brain_region']=='HPC-PFC'],
            y = 'PCo',x='condition', hue = 'drug', ax= ax, dodge = True, color = 'black')
ax.legend([],[], frameon=False)

plt.savefig(r'G:\My Drive\Fenton_Lab\Ketamine_analysis\Manuscript\PCo_barplot_HPC_PFC' + '.pdf', transparent = True)
plt.show()
plt.clf()
plt.close()  

In [ ]:
df_sal_rsk_PCo_normchange = []
ls_tmp = []
## Kendall tau correlations - all times - cell pair correlations

for it in range(0,len(mouse_reduced_list)):
    
    if np.sum(np.isnan(np.array(PCo_all_mov[it][0]))) == 3:
        continue

    ### create dataframe saving PCo values
    ls_tmp.append(pd.DataFrame({'drug':drug_reduced_list[it], 'condition': ['dose3/1'],
                                'PCo':np.array(PCo_all_mov[it][0])[:,0][-1]-np.array(PCo_all_mov[it][0])[:,0][0],'brain_region': 'HPC-HPC',
                                'mouse': mouse_reduced_list[it]}))

    ls_tmp.append(pd.DataFrame({'drug':drug_reduced_list[it], 'condition': ['dose3/1'],
                                'PCo':np.array(PCo_all_mov[it][1])[:,0][-1]-np.array(PCo_all_mov[it][1])[:,0][0],'brain_region': 'PFC-PFC',
                                'mouse': mouse_reduced_list[it]}))

    ls_tmp.append(pd.DataFrame({'drug':drug_reduced_list[it], 'condition': ['dose3/1'],
                                'PCo':np.array(PCo_all_mov[it][2])[:,0][-1]-np.array(PCo_all_mov[it][2])[:,0][0],'brain_region': 'HPC-PFC',
                                'mouse': mouse_reduced_list[it]}))    
   

In [ ]:
df_sal_rsk_PCo_normchange = pd.concat(ls_tmp)
df_sal_rsk_PCo_normchange.head(4)

In [ ]:
## PCo saline rsk

fig = plt.figure(figsize=(3,3))
ax = fig.add_subplot(1,1,1)

sns.barplot(data=df_sal_rsk_PCo_normchange[df_sal_rsk_PCo_normchange['brain_region']=='HPC-HPC'],
            y = 'PCo',x='condition', hue = 'drug', ax= ax, errorbar = 'se')

sns.stripplot(data=df_sal_rsk_PCo_normchange[df_sal_rsk_PCo_normchange['brain_region']=='HPC-HPC'],
            y = 'PCo',x='condition', hue = 'drug', ax= ax, dodge = True, color = 'black')
ax.legend([],[], frameon=False)

plt.savefig(r'G:\My Drive\Fenton_Lab\Ketamine_analysis\Manuscript\PCo_barplot_HPC_v2' + '.pdf', transparent = True)
plt.show()
plt.clf()
plt.close()  

In [ ]:
## PCo saline rsk

fig = plt.figure(figsize=(3,3))
ax = fig.add_subplot(1,1,1)

sns.barplot(data=df_sal_rsk_PCo_normchange[df_sal_rsk_PCo_normchange['brain_region']=='PFC-PFC'],
            y = 'PCo',x='condition', hue = 'drug', ax= ax, errorbar = 'se')

sns.stripplot(data=df_sal_rsk_PCo_normchange[df_sal_rsk_PCo_normchange['brain_region']=='PFC-PFC'],
            y = 'PCo',x='condition', hue = 'drug', ax= ax, dodge = True, color = 'black')
ax.legend([],[], frameon=False)

plt.savefig(r'G:\My Drive\Fenton_Lab\Ketamine_analysis\Manuscript\PCo_barplot_PFC_v2' + '.pdf', transparent = True)
plt.show()
plt.clf()
plt.close()  

In [ ]:
## PCo saline rsk

fig = plt.figure(figsize=(3,3))
ax = fig.add_subplot(1,1,1)

sns.barplot(data=df_sal_rsk_PCo_normchange[df_sal_rsk_PCo_normchange['brain_region']=='HPC-PFC'],
            y = 'PCo',x='condition', hue = 'drug', ax= ax, errorbar = 'se')

sns.stripplot(data=df_sal_rsk_PCo_normchange[df_sal_rsk_PCo_normchange['brain_region']=='HPC-PFC'],
            y = 'PCo',x='condition', hue = 'drug', ax= ax, dodge = True, color = 'black')
ax.legend([],[], frameon=False)

plt.savefig(r'G:\My Drive\Fenton_Lab\Ketamine_analysis\Manuscript\PCo_barplot_HPC_PFC_v2' + '.pdf', transparent = True)
plt.show()
plt.clf()
plt.close()  

In [ ]:
def tau_distribution_timeseries(tau_data,mask_brain):

    kendall_timeseries = []
    kendall_timeseries = tau_data.copy()
    ### plot distributions of tau in time
    bins = np.arange(-1,1,0.02)
    
    ratio_pos_neg_hpc = []
    ratio_overall_hpc = []
    
    fig, axlist = plt.subplots(1, len(kendall_timeseries), sharex=True, sharey=True, figsize = (20,4))
    for isplit,ax in enumerate(axlist):        
        
        tau_, pval, pos_idx, neg_idx = [], [], [], []
        tau_ = np.hstack(kendall_timeseries[isplit])[:,0][mask_brain[0]]
        pval = np.hstack(kendall_timeseries[isplit])[:,1][mask_brain[0]]

        pos_idx = (tau_ > 0)*(pval < 0.05)
        neg_idx = (tau_ < 0)*(pval < 0.05)

        sns.histplot(tau_[pos_idx], bins = bins, ax = ax, color = 'red')
        sns.histplot(tau_[neg_idx], bins = bins, ax = ax, color = 'blue')
#         ax.set_xlim(-1,1)
#         ax.set_ylim(200)
        ax.axis(False)
    
        ratio_pos_neg_hpc.append(np.sum(neg_idx) / np.sum(pos_idx))
        ratio_overall_hpc.append(np.sum(pval < 0.05) / pval.size)
    
        #     ax.set_title(str(np.nanmean(tau_)))
    plt.title('HPC-HPC pair of cells')
    plt.show()

    
    fig2 = plt.figure(figsize = (6,6))
    ax2 = fig2.add_subplot(1,1,1)
    ax2.plot(range(len(ratio_pos_neg_hpc)), ratio_pos_neg_hpc, color = 'green')
    ax2.plot(range(len(ratio_pos_neg_hpc)), ratio_overall_hpc, color = 'gray')
    plt.show()    
    
    
    ratio_pos_neg_pfc = []
    ratio_overall_pfc = []
    
    fig, axlist = plt.subplots(1, len(kendall_timeseries), sharex=True, sharey=True, figsize = (20,4))
    for isplit,ax in enumerate(axlist):        
        
        tau_, pval, pos_idx, neg_idx = [], [], [], []
        tau_ = np.hstack(kendall_timeseries[isplit])[:,0][mask_brain[1]]
        pval = np.hstack(kendall_timeseries[isplit])[:,1][mask_brain[1]]

        pos_idx = (tau_ > 0)*(pval < 0.05)
        neg_idx = (tau_ < 0)*(pval < 0.05)

        sns.histplot(tau_[pos_idx], bins = bins, ax = ax, color = 'red')
        sns.histplot(tau_[neg_idx], bins = bins, ax = ax, color = 'blue')
#         ax.set_xlim(-1,1)
#         ax.set_ylim(200)
        ax.axis(False)
    
        ratio_pos_neg_pfc.append(np.sum(neg_idx) / np.sum(pos_idx))
        ratio_overall_pfc.append(np.sum(pval < 0.05) / pval.size)    
    
    plt.title('mPFC-mPFC pair of cells')
    plt.show()
    
    
    fig2 = plt.figure(figsize = (6,6))
    ax2 = fig2.add_subplot(1,1,1)
    ax2.plot(range(len(ratio_pos_neg_pfc)), ratio_pos_neg_pfc, color = 'orange')
    ax2.plot(range(len(ratio_pos_neg_pfc)), ratio_overall_pfc, color = 'gray')
    plt.show()

    ratio_pos_neg_hpcpfc = []
    ratio_overall_hpcpfc = []
    
    fig, axlist = plt.subplots(1, len(kendall_timeseries), sharex=True, sharey=True, figsize = (20,4))
    for isplit,ax in enumerate(axlist):        
        
        tau_, pval, pos_idx, neg_idx = [], [], [], []
        tau_ = np.hstack(kendall_timeseries[isplit])[:,0][mask_brain[2]]
        pval = np.hstack(kendall_timeseries[isplit])[:,1][mask_brain[2]]

        pos_idx = (tau_ > 0)*(pval < 0.05)
        neg_idx = (tau_ < 0)*(pval < 0.05)

        sns.histplot(tau_[pos_idx], bins = bins, ax = ax, color = 'red')
        sns.histplot(tau_[neg_idx], bins = bins, ax = ax, color = 'blue')
#         ax.set_xlim(-1,1)
#         ax.set_ylim(200)
        ax.axis(False)
    
        ratio_pos_neg_hpcpfc.append(np.sum(neg_idx) / np.sum(pos_idx))
        ratio_overall_hpcpfc.append(np.sum(pval < 0.05) / pval.size)      
    
        #     ax.set_title(str(np.nanmean(tau_)))
    plt.title('mPFC-HPC pair of cells')
    plt.show()
    
    fig2 = plt.figure(figsize = (6,6))
    ax2 = fig2.add_subplot(1,1,1)
    ax2.plot(range(len(ratio_pos_neg_hpcpfc)), ratio_pos_neg_hpcpfc, color = 'brown')
    ax2.plot(range(len(ratio_pos_neg_hpcpfc)), ratio_overall_hpcpfc, color = 'gray')
    plt.show()
    
    
    return ratio_pos_neg_hpc, ratio_pos_neg_pfc, ratio_pos_neg_hpcpfc, ratio_overall_hpc, ratio_overall_pfc, ratio_overall_hpcpfc

In [ ]:
split = 10*60*10

In [ ]:
## distribution of moving points
kendall_timeseries_all = []

for irec in range(0,len(x_aln)):

    if (drug_list[irec] == 'SAL') or (drug_list[irec] == 'RSK'):

        ksort_path = ksort_HPC_path_matched[irec]
        print('experiment',ksort_path)
        size_to = np.min([dose3, mask_mov_all[irec].size])
        
        mov_points = []
        mov_points = [np.sum(mask_mov_all[irec][s*split:(s+1)*split]) for s in  range(0,(mask_mov_all[irec].size // split)-1)]
        
        kendall_timeseries_tmp = []
        kendall_timeseries_tmp = [kendall_tau_parallel(spikes_binned_HPC_PFC[irec][:,s*split:(s+1)*split][:,mask_mov_all[irec][s*split:(s+1)*split]])[2] for s in range(0,(mask_mov_all[irec].size // split))]

        cellpairs_baseline = kendall_tau_parallel(spikes_binned_HPC_PFC[irec][:,0:10])[1]
        
       # mask for pairs of cells from HPC-HPC / PFC-PFC / HPC-PFC
        HPC_idx = np.arange(0,spikes_binned_all_HPC[irec].shape[0])  #Change index depending of the recording - change the variable names

        test = []
        test = [np.intersect1d(HPC_idx,np.hstack(p)).size for p in cellpairs_baseline]

        mask_HPC_HPC = np.zeros(len(cellpairs_baseline)).astype(bool)
        mask_HPC_HPC[np.hstack(test)==2] = True

        mask_HPC_PFC = np.zeros(len(cellpairs_baseline)).astype(bool)
        mask_HPC_PFC[np.hstack(test)==1] = True

        mask_PFC_PFC = np.zeros(len(cellpairs_baseline)).astype(bool)
        mask_PFC_PFC[np.hstack(test)==0] = True        
        
        print('Distribution of moving points',np.hstack(mov_points))
        
        ## plot distribution in time
        
        tau_distribution_timeseries(kendall_timeseries_tmp,[mask_HPC_HPC,mask_PFC_PFC,mask_HPC_PFC])
        
        kendall_timeseries_all.append([kendall_timeseries_tmp,[mask_HPC_HPC,mask_PFC_PFC,mask_HPC_PFC]])

In [ ]:
# # to save "processed" tau timeseries
# with open(r'G:\My Drive\Fenton_Lab\Ketamine_analysis\Ketamine_data\Neurotar_HPC_ketamine\tauvec_timeseries_10min.file', 'wb') as f:
#     pickle.dump(kendall_timeseries_all, f, pickle.HIGHEST_PROTOCOL)
# print('saved!')

In [ ]:
# load data
kendall_timeseries_all_mov = []
with open(r'G:\My Drive\Fenton_Lab\Ketamine_analysis\Ketamine_data\Neurotar_HPC_ketamine\tauvec_timeseries_10min_moving.file', 'rb') as f3:
    kendall_timeseries_all_mov = pickle.load(f3)    
print('loaded!')

In [ ]:
drug_reduced_list = []
mouse_reduced_list = []

for irec in range(0,len(x_aln)):

    if (drug_list[irec] == 'SAL') or (drug_list[irec] == 'RSK'):

        ksort_path = ksort_HPC_path_matched[irec]
        print('experiment',ksort_path)

        drug_reduced_list.append(drug_list[irec])
        mouse_reduced_list.append(mouse_list[irec])


## Visualize tau vector changes in time

In [ ]:
mask_brain_all = []

for irec in range(0,len(x_aln)):

    if (drug_list[irec] == 'SAL') or (drug_list[irec] == 'RSK'):

        ksort_path = ksort_HPC_path_matched[irec]
        print('experiment',ksort_path)

        cellpairs_baseline = kendall_tau_parallel(spikes_binned_HPC_PFC[irec][:,0:10])[1]
        
       # mask for pairs of cells from HPC-HPC / PFC-PFC / HPC-PFC
        HPC_idx = np.arange(0,spikes_binned_all_HPC[irec].shape[0])  #Change index depending of the recording - change the variable names

        test = []
        test = [np.intersect1d(HPC_idx,np.hstack(p)).size for p in cellpairs_baseline]

        mask_HPC_HPC = np.zeros(len(cellpairs_baseline)).astype(bool)
        mask_HPC_HPC[np.hstack(test)==2] = True

        mask_HPC_PFC = np.zeros(len(cellpairs_baseline)).astype(bool)
        mask_HPC_PFC[np.hstack(test)==1] = True

        mask_PFC_PFC = np.zeros(len(cellpairs_baseline)).astype(bool)
        mask_PFC_PFC[np.hstack(test)==0] = True     
        
        mask_brain_all.append([mask_HPC_HPC,mask_PFC_PFC,mask_HPC_PFC])

In [ ]:
mask_brain_all[irec]

In [ ]:
ratios_all = []
ls_tmp = []

for irec in range(0,len(kendall_timeseries_all_mov)):
    
    l1,l2,l3,l4,l5,l6 = [],[],[],[],[],[]
    l1,l2,l3,l4,l5,l6 = tau_distribution_timeseries(kendall_timeseries_all_mov[irec][0],mask_brain_all[irec])
    
    ratios_all.append([l1,l2,l3,l4,l5,l6])
    
     ### create dataframe saving PCo values
    ls_tmp.append(pd.DataFrame({'drug':drug_reduced_list[irec], 'condition': range(len(l1)),
                                'PCo':l1,'brain_region': 'HPC-HPC',
                                'mouse': mouse_reduced_list[irec]}))

    ls_tmp.append(pd.DataFrame({'drug':drug_reduced_list[irec], 'condition': range(len(l1)),
                                'PCo':l2,'brain_region': 'PFC-PFC',
                                'mouse': mouse_reduced_list[irec]}))

    ls_tmp.append(pd.DataFrame({'drug':drug_reduced_list[irec], 'condition': range(len(l1)),
                                'PCo':l3,'brain_region': 'HPC-PFC',
                                'mouse': mouse_reduced_list[irec]}))       

In [ ]:
df_tau_ratio = pd.concat(ls_tmp)
df_tau_ratio.head(10)

In [ ]:
fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot(1,1,1)

sns.lineplot(data = df_tau_ratio.reset_index()[df_tau_ratio.reset_index()['drug']=='SAL'], x = 'condition',
             y = 'PCo', hue = 'brain_region')
plt.ylabel('Neg/Pos ratio')
plt.show()

In [ ]:
fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot(1,1,1)

sns.lineplot(data = df_tau_ratio.reset_index()[df_tau_ratio.reset_index()['drug']=='RSK'], x = 'condition',
             y = 'PCo', hue = 'brain_region')
plt.ylabel('Neg/Pos ratio')
plt.show()

In [ ]:
## distribution of sta points
kendall_timeseries_all = []

for irec in range(0,len(x_aln)):

    if (drug_list[irec] == 'SAL') or (drug_list[irec] == 'RSK'):

        ksort_path = ksort_HPC_path_matched[irec]
        print('experiment',ksort_path)
        size_to = np.min([dose3, mask_mov_all[irec].size])
        
        mov_points = []
        mov_points = [np.sum(mask_sta_all[irec][s*split:(s+1)*split]) for s in  range(0,(mask_sta_all[irec].size // split)-1)]
        
        kendall_timeseries_tmp = []
        kendall_timeseries_tmp = [kendall_tau_parallel(spikes_binned_HPC_PFC[irec][:,s*split:(s+1)*split][:,mask_sta_all[irec][s*split:(s+1)*split]])[2] for s in range(0,(mask_sta_all[irec].size // split))]

        cellpairs_baseline = kendall_tau_parallel(spikes_binned_HPC_PFC[irec][:,0:10])[1]
        
       # mask for pairs of cells from HPC-HPC / PFC-PFC / HPC-PFC
        HPC_idx = np.arange(0,spikes_binned_all_HPC[irec].shape[0])  #Change index depending of the recording - change the variable names

        test = []
        test = [np.intersect1d(HPC_idx,np.hstack(p)).size for p in cellpairs_baseline]

        mask_HPC_HPC = np.zeros(len(cellpairs_baseline)).astype(bool)
        mask_HPC_HPC[np.hstack(test)==2] = True

        mask_HPC_PFC = np.zeros(len(cellpairs_baseline)).astype(bool)
        mask_HPC_PFC[np.hstack(test)==1] = True

        mask_PFC_PFC = np.zeros(len(cellpairs_baseline)).astype(bool)
        mask_PFC_PFC[np.hstack(test)==0] = True        
        
        print('Distribution of stationary points',np.hstack(sta_points))
        
        ## plot distribution in time
        
        tau_distribution_timeseries(kendall_timeseries_tmp,[mask_HPC_HPC,mask_PFC_PFC,mask_HPC_PFC])
        
        kendall_timeseries_all.append([kendall_timeseries_tmp,[mask_HPC_HPC,mask_PFC_PFC,mask_HPC_PFC]])

In [ ]:
# # to save "processed" tau timeseries
# with open(r'G:\My Drive\Fenton_Lab\Ketamine_analysis\Ketamine_data\Neurotar_HPC_ketamine\tauvec_timeseries_10min_stationary.file', 'wb') as f:
#     pickle.dump(kendall_timeseries_all, f, pickle.HIGHEST_PROTOCOL)
# print('saved!')

In [ ]:
# load data
kendall_timeseries_all_sta = []
with open(r'G:\My Drive\Fenton_Lab\Ketamine_analysis\Ketamine_data\Neurotar_HPC_ketamine\tauvec_timeseries_10min_stationary.file', 'rb') as f3:
    kendall_timeseries_all_sta = pickle.load(f3)    
print('loaded!')

In [ ]:
ratios_all = []
ls_tmp = []

for irec in range(0,len(kendall_timeseries_all_sta)):
    
    l1,l2,l3,l4,l5,l6 = [],[],[],[],[],[]
    l1,l2,l3,l4,l5,l6 = tau_distribution_timeseries(kendall_timeseries_all_sta[irec][0],mask_brain_all[irec])
    
    ratios_all.append([l1,l2,l3,l4,l5,l6])
    
     ### create dataframe saving PCo values
    ls_tmp.append(pd.DataFrame({'drug':drug_reduced_list[irec], 'condition': range(len(l1)),
                                'PCo':l1,'brain_region': 'HPC-HPC',
                                'mouse': mouse_reduced_list[irec]}))

    ls_tmp.append(pd.DataFrame({'drug':drug_reduced_list[irec], 'condition': range(len(l1)),
                                'PCo':l2,'brain_region': 'PFC-PFC',
                                'mouse': mouse_reduced_list[irec]}))

    ls_tmp.append(pd.DataFrame({'drug':drug_reduced_list[irec], 'condition': range(len(l1)),
                                'PCo':l3,'brain_region': 'HPC-PFC',
                                'mouse': mouse_reduced_list[irec]}))       

In [ ]:
df_tau_ratio_sta = pd.concat(ls_tmp)
df_tau_ratio_sta.head(10)

In [ ]:
fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot(1,1,1)

sns.lineplot(data = df_tau_ratio_sta.reset_index()[df_tau_ratio_sta.reset_index()['drug']=='SAL'], x = 'condition',
             y = 'PCo', hue = 'brain_region')
plt.ylabel('Neg/Pos ratio')
plt.show()

In [ ]:
fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot(1,1,1)

sns.lineplot(data = df_tau_ratio_sta.reset_index()[df_tau_ratio_sta.reset_index()['drug']=='RSK'], x = 'condition',
             y = 'PCo', hue = 'brain_region')
plt.ylabel('Neg/Pos ratio')
plt.show()

# Positive, negative, uncorrelated dynamics

In [ ]:
# load data
tauVec_all_sta, tauVec_all_mov, tauVec_all_baseline_halves_sta, tauVec_all_baseline_halves_mov = [],[],[],[]
with open(r'G:\My Drive\Fenton_Lab\Ketamine_analysis\Ketamine_data\Neurotar_HPC_ketamine\corr_mov_sta_Ketamine.file', 'rb') as f3:
    [tauVec_all_sta, tauVec_all_mov, tauVec_all_baseline_halves_sta, tauVec_all_baseline_halves_mov] = pickle.load(f3)    
print('loaded!')

In [ ]:
# load data
tauVec_all_, tauVec_all_baseline_halves_alltimes = [], []
with open(r'G:\My Drive\Fenton_Lab\Ketamine_analysis\Ketamine_data\Neurotar_HPC_ketamine\corr_alltimes_Ketamine.file', 'rb') as f3:
    [tauVec_all_, tauVec_all_baseline_halves_alltimes] = pickle.load(f3)    
print('loaded!')

In [ ]:
## Kendall tau correlations - all times - cell pair correlations
drug_reduced_list = []
mouse_reduced_list = []

for irec in range(0,len(x_aln)):

    if (drug_list[irec] == 'SAL') or (drug_list[irec] == 'RSK'):

        ksort_path = ksort_HPC_path_matched[irec]
        print('experiment',ksort_path)
        size_to = np.min([dose3, mask_mov_all[irec].size])

        drug_reduced_list.append(drug_list[irec])
        mouse_reduced_list.append(mouse_list[irec])


In [ ]:
def tau_corr_dynamics(tau_data,mask_brain):

    kendall_timeseries = []
    kendall_timeseries = tau_data.copy()
    ### plot distributions of tau in time
    bins = np.arange(-1,1,0.02)
    
    ## "tagged" cell pairs according to their firing during baseline
    
    tau_, pval, pos_idx, neg_idx, ind_idx = [], [], [], [], []
    tau_ = np.nan_to_num(np.hstack(kendall_timeseries[0])[:,0][mask_brain])
    pval = np.nan_to_num(np.hstack(kendall_timeseries[0])[:,1][mask_brain])

    pos_idx = (tau_ >= 0)*(pval < 0.05)
    neg_idx = (tau_ < 0)*(pval < 0.05)  
    ind_idx = (pval >= 0.05)  
    
    tau_dynamics = []
    ## track the distribution of pos, neg and ind across conditions

    for icond in range(0,4):
        
        tau_, pval = [], []
        tau_ = np.nan_to_num(np.hstack(kendall_timeseries[icond])[:,0][mask_brain])
        pval = np.nan_to_num(np.hstack(kendall_timeseries[icond])[:,1][mask_brain])  
        
        pos_change = [np.sum((tau_[pos_idx] >= 0)*(pval[pos_idx] < 0.05))/tau_[pos_idx].size,
                      np.sum((pval[pos_idx] >= 0.05))/tau_[pos_idx].size,
                      np.sum((tau_[pos_idx] < 0)*(pval[pos_idx] < 0.05))/tau_[pos_idx].size]

        neg_change = [np.sum((tau_[neg_idx] >= 0)*(pval[neg_idx] < 0.05))/tau_[neg_idx].size,
                      np.sum((pval[neg_idx] >= 0.05))/tau_[neg_idx].size,
                      np.sum((tau_[neg_idx] < 0)*(pval[neg_idx] < 0.05))/tau_[neg_idx].size]  
        
        ind_change = [np.sum((tau_[ind_idx] >= 0)*(pval[ind_idx] < 0.05))/tau_[ind_idx].size,
                      np.sum((pval[ind_idx] >= 0.05))/tau_[ind_idx].size,
                      np.sum((tau_[ind_idx] < 0)*(pval[ind_idx] < 0.05))/tau_[ind_idx].size]    
        
        
        tau_dynamics.append([pos_change,ind_change,neg_change])

    return tau_dynamics

In [ ]:
tau_change_all_hpc = []
tau_change_all_pfc = []
tau_change_all_hpcpfc = []

tau_change_mov_hpc = []
tau_change_mov_pfc = []
tau_change_mov_hpcpfc = []

tau_change_sta_hpc = []
tau_change_sta_pfc = []
tau_change_sta_hpcpfc = []

for irec in range(0,len(x_aln)):

    if (drug_list[irec] == 'SAL') or (drug_list[irec] == 'RSK'):

        ksort_path = ksort_HPC_path_matched[irec]
        print('experiment',ksort_path)

        cellpairs_baseline = kendall_tau_parallel(spikes_binned_HPC_PFC[irec][:,0:10])[1]
        
       # mask for pairs of cells from HPC-HPC / PFC-PFC / HPC-PFC
        HPC_idx = np.arange(0,spikes_binned_all_HPC[irec].shape[0])  #Change index depending of the recording - change the variable names

        test = []
        test = [np.intersect1d(HPC_idx,np.hstack(p)).size for p in cellpairs_baseline]

        mask_HPC_HPC = np.zeros(len(cellpairs_baseline)).astype(bool)
        mask_HPC_HPC[np.hstack(test)==2] = True

        mask_HPC_PFC = np.zeros(len(cellpairs_baseline)).astype(bool)
        mask_HPC_PFC[np.hstack(test)==1] = True

        mask_PFC_PFC = np.zeros(len(cellpairs_baseline)).astype(bool)
        mask_PFC_PFC[np.hstack(test)==0] = True     
        
        mask_brain_tmp = [mask_HPC_HPC,mask_PFC_PFC,mask_HPC_PFC]
        
        ## moving 
        tau_change = []
        tau_change = tau_corr_dynamics(tauVec_all_mov[irec],mask_HPC_HPC)
        tau_change_mov_hpc.append(tau_change)

        tau_change = []
        tau_change = tau_corr_dynamics(tauVec_all_mov[irec],mask_PFC_PFC)
        tau_change_mov_pfc.append(tau_change)
        
        tau_change = []
        tau_change = tau_corr_dynamics(tauVec_all_mov[irec],mask_HPC_PFC)
        tau_change_mov_hpcpfc.append(tau_change)        

        ## stationary
        tau_change = []
        tau_change = tau_corr_dynamics(tauVec_all_sta[irec],mask_HPC_HPC)
        tau_change_sta_hpc.append(tau_change)

        tau_change = []
        tau_change = tau_corr_dynamics(tauVec_all_sta[irec],mask_PFC_PFC)
        tau_change_sta_pfc.append(tau_change)
        
        tau_change = []
        tau_change = tau_corr_dynamics(tauVec_all_sta[irec],mask_HPC_PFC)
        tau_change_sta_hpcpfc.append(tau_change) 
        
        ## all timepoints
        tau_change = []
        tau_change = tau_corr_dynamics(tauVec_all_[irec],mask_HPC_HPC)
        tau_change_all_hpc.append(tau_change)

        tau_change = []
        tau_change = tau_corr_dynamics(tauVec_all_[irec],mask_PFC_PFC)
        tau_change_all_pfc.append(tau_change)
        
        tau_change = []
        tau_change = tau_corr_dynamics(tauVec_all_[irec],mask_HPC_PFC)
        tau_change_all_hpcpfc.append(tau_change)             
        

In [ ]:
def piechart_plot(raw_data):

    for icond in range(0,4):

        try:
            data = raw_data[icond].copy()
            labels = ['Pos.', 'Ind.', 'Neg.']

            #define Seaborn color palette to use
            colors = ['red', "gray", "blue"] #sns.color_palette('pastel')[0:5]

            fig = plt.figure(figsize=(12,12))
            ax = fig.add_subplot(4,3,1 + 3*icond)
            #create pie chart
            
            if icond > 0:
                ax.pie(data[0], labels = labels, colors = colors, autopct='%.0f%%')
            if icond == 0:
                ax.pie(data[0], labels = labels, colors = colors, autopct='%.0f%%',labeldistance=None)

            ax2 = fig.add_subplot(4,3,2+ 3*icond)
            #create pie chart
            ax2.pie(data[1], labels = labels, colors = colors, autopct='%.0f%%',labeldistance=None)

            ax3 = fig.add_subplot(4,3,3+ 3*icond)
            #create pie chart
            ax3.pie(data[2], labels = labels, colors = colors, autopct='%.0f%%',labeldistance=None)

            plt.show()
        except:
            continue
    return

In [ ]:
for irec in range(0,6): #len(tau_change_all_hpc)):
    
    print(mouse_reduced_list[irec],drug_reduced_list[irec])
    
    print('Hippocampal cell pairs')
    piechart_plot(tau_change_all_hpc[irec])
    
    print('mPFC cell pairs')
    piechart_plot(tau_change_all_pfc[irec])    
    
    print('mPFC-HPC cell pairs')
    piechart_plot(tau_change_all_hpcpfc[irec])        

In [ ]:
for irec in range(0,6): #len(tau_change_all_hpc)):
    
    print(mouse_reduced_list[irec],drug_reduced_list[irec])
    
    print('Hippocampal cell pairs')
    piechart_plot(tau_change_mov_hpc[irec])
    
    print('mPFC cell pairs')
    piechart_plot(tau_change_mov_pfc[irec])    
    
    print('mPFC-HPC cell pairs')
    piechart_plot(tau_change_mov_hpcpfc[irec])        

In [ ]:
for irec in range(0,6): #len(tau_change_all_hpc)):
    
    print(mouse_reduced_list[irec],drug_reduced_list[irec])
    
    print('Hippocampal cell pairs')
    piechart_plot(tau_change_sta_hpc[irec])
    
    print('mPFC cell pairs')
    piechart_plot(tau_change_sta_pfc[irec])    
    
    print('mPFC-HPC cell pairs')
    piechart_plot(tau_change_sta_hpcpfc[irec])    

## AV correlation matrices

In [ ]:
## binned spiking activity from 1 second resolution to 1 minute (60 samples at a time)

## change to 10 seconds, 30 seconds, 60 seconds, 120 seconds, 300 seconds
bin_size = 60 ## 60 seconds

spk_binned_60s_all_HPC = []
spk_binned_60s_all_PFC = []

for irec in range(0,len(x)):
    print(ipath)

    ## HPC probe
    bins = spikes_binned_all_HPC[irec].shape[1] / bin_size
    
    spk_tmp = []
    spk_tmp = getMeanBinnedVector_2d(spikes_binned_all_HPC[irec].T, bins, return_format="array")
    print(spk_tmp.shape)
    
    spk_binned_60s_all_HPC.append(spk_tmp)
    spk_tmp = []
    

    ## mPFC probe
    bins = spikes_binned_all_PFC[irec].shape[1] / bin_size
    
    spk_tmp = []
    spk_tmp = getMeanBinnedVector_2d(spikes_binned_all_PFC[irec].T, bins, return_format="array")
    print(spk_tmp.shape)
    
    spk_binned_60s_all_PFC.append(spk_tmp)
    spk_tmp = []    
    

In [ ]:
## AV matrix computation

for irec in range(0,len(x_aln)):

    
    ksort_path = ksort_HPC_path_matched[irec]
    print('experiment',ksort_path)
    
    corr_AV_HPC, cellpairs_AV_HPC, tauVec_AV_HPC = [],[],[]
    corr_AV_HPC, cellpairs_AV_HPC, tauVec_AV_HPC = pearson_parallel(spk_binned_60s_all_HPC[irec][mask_fr_HPC_all[irec],:].T)
    
    print('maybe principal cells', spk_binned_60s_all_HPC[irec][mask_fr_HPC_all[irec],:].shape[0],
          'of total',spk_binned_60s_all_HPC[irec].shape[0])
    
    corr_AV_PFC, cellpairs_AV_PFC, tauVec_AV_PFC = [],[],[]
    corr_AV_PFC, cellpairs_AV_PFC, tauVec_AV_PFC = pearson_parallel(spk_binned_60s_all_PFC[irec].T)    
    
    # Correlation matrix visualization
    
    fig = plt.figure(figsize=(12,6))
    
    ax = fig.add_subplot(1,2,1)
    sns.heatmap(corr_AV_HPC, cmap = 'jet', ax=ax, cbar_kws={'label': 'Correlation (r)'})
    ## lines to separate injections
    ax.plot(np.ones(corr_AV_HPC.shape[0])*30, np.arange(0,corr_AV_HPC.shape[0]), '--', color = 'white')
    ax.plot(np.arange(0,corr_AV_HPC.shape[0]), np.ones(corr_AV_HPC.shape[0])*30, '--', color = 'white')
    ax.plot(np.ones(corr_AV_HPC.shape[0])*60, np.arange(0,corr_AV_HPC.shape[0]), '--', color = 'white')
    ax.plot(np.arange(0,corr_AV_HPC.shape[0]), np.ones(corr_AV_HPC.shape[0])*60, '--', color = 'white')
    ax.plot(np.ones(corr_AV_HPC.shape[0])*90, np.arange(0,corr_AV_HPC.shape[0]), '--', color = 'white')
    ax.plot(np.arange(0,corr_AV_HPC.shape[0]), np.ones(corr_AV_HPC.shape[0])*90, '--', color = 'white')
    ax.plot(np.ones(corr_AV_HPC.shape[0])*120, np.arange(0,corr_AV_HPC.shape[0]), '--', color = 'white')
    ax.plot(np.arange(0,corr_AV_HPC.shape[0]), np.ones(corr_AV_HPC.shape[0])*120, '--', color = 'white')
    ax.set_title('HPC')
#     ax.axis('equal')
    ax.set_xlabel('Time [min]')
    ax.set_ylabel('Time [min]')
    
    ax2 = fig.add_subplot(1,2,2)
    sns.heatmap(corr_AV_PFC, cmap = 'jet', ax= ax2, cbar_kws={'label': 'Correlation (r)'})
    ## lines to separate injections
    ax2.plot(np.ones(corr_AV_HPC.shape[0])*30, np.arange(0,corr_AV_HPC.shape[0]), '--', color = 'white')
    ax2.plot(np.arange(0,corr_AV_HPC.shape[0]), np.ones(corr_AV_HPC.shape[0])*30, '--', color = 'white')
    ax2.plot(np.ones(corr_AV_HPC.shape[0])*60, np.arange(0,corr_AV_HPC.shape[0]), '--', color = 'white')
    ax2.plot(np.arange(0,corr_AV_HPC.shape[0]), np.ones(corr_AV_HPC.shape[0])*60, '--', color = 'white')
    ax2.plot(np.ones(corr_AV_HPC.shape[0])*90, np.arange(0,corr_AV_HPC.shape[0]), '--', color = 'white')
    ax2.plot(np.arange(0,corr_AV_HPC.shape[0]), np.ones(corr_AV_HPC.shape[0])*90, '--', color = 'white')
    ax2.plot(np.ones(corr_AV_HPC.shape[0])*120, np.arange(0,corr_AV_HPC.shape[0]), '--', color = 'white')
    ax2.plot(np.arange(0,corr_AV_HPC.shape[0]), np.ones(corr_AV_HPC.shape[0])*120, '--', color = 'white')
    ax2.set_title('mPFC')
    ax2.set_xlabel('Time [min]')
    ax2.set_ylabel('Time [min]')  
#     ax2.axis('equal')
    plt.tight_layout()
    plt.show()
    

# Manifold analysis

## ISOMAP projections

In [ ]:
dim_to = 10

ISO_all_HPC = []
ISO_all_PFC = []

for irec in range(0,len(x)):
     
    ksort_path = ksort_HPC_path_matched[irec]
    print('experiment',ksort_path)        
    
    if spikes_binned_all_HPC[irec].shape[0] > 10:

        if  spikes_binned_all_HPC[irec].shape[1] > 7201:
            dose_color = []
            dose_color = np.hstack([np.ones(baseline), 2*np.ones(baseline), 3*np.ones(baseline),
                                   4*np.ones(baseline), 5*np.ones(spikes_binned_all_HPC[irec].shape[1]-4*60*30)])       

        if  spikes_binned_all_HPC[irec].shape[1] <= 7201:
            dose_color = []
            dose_color = np.hstack([np.ones(baseline), 2*np.ones(baseline), 3*np.ones(baseline),
                                   4*np.ones(spikes_binned_all_HPC[irec].shape[1]-3*60*30)])    

        print('HPC')    
        tmp_iso = []
        tmp_iso = isomap_proj(dose_color, spikes_binned_all_HPC[irec], dim_to, 100)
        ISO_all_HPC.append(tmp_iso)

        print('mPFC')        
        tmp_iso = []
        tmp_iso = isomap_proj(dose_color, spikes_binned_all_PFC[irec], dim_to, 100)    
        ISO_all_PFC.append(tmp_iso)    

## 3d ISomap + AV matrix + PCo scatterplot

In [ ]:
irec_real = 0
for irec in range(0,len(x)):
    
    ksort_path = ksort_HPC_path_matched[irec]
    print('experiment',ksort_path)
    
    if spikes_binned_all_HPC[irec].shape[0] > 10:
        
        if  spikes_binned_all_HPC[irec].shape[1] > 7201:
            dose_color = []
            dose_color = np.hstack([np.ones(baseline), 2*np.ones(baseline), 3*np.ones(baseline),
                                   4*np.ones(baseline), 5*np.ones(spikes_binned_all_HPC[irec].shape[1]-4*60*30)])       

        if  spikes_binned_all_HPC[irec].shape[1] <= 7201:
            dose_color = []
            dose_color = np.hstack([np.ones(baseline), 2*np.ones(baseline), 3*np.ones(baseline),
                                   4*np.ones(spikes_binned_all_HPC[irec].shape[1]-3*60*30)])        
    
    
        corr_AV_HPC, cellpairs_AV_HPC, tauVec_AV_HPC = [],[],[]
        corr_AV_HPC, cellpairs_AV_HPC, tauVec_AV_HPC = pearson_parallel(spk_binned_60s_all_HPC[irec][mask_fr_HPC_all[irec],:].T)

        corr_AV_PFC, cellpairs_AV_PFC, tauVec_AV_PFC = [],[],[]
        corr_AV_PFC, cellpairs_AV_PFC, tauVec_AV_PFC = pearson_parallel(spk_binned_60s_all_PFC[irec].T)    


        fig = plt.figure(figsize=(10,10))

        # ISomap HPC
        ax = fig.add_subplot(2,2,1, projection = '3d')
        tmp_sim = []
        tmp_sim = ISO_all_HPC[irec_real].copy()
        p3 = ax.scatter(tmp_sim[:, 0], tmp_sim[:, 1], tmp_sim[:, 2], c=dose_color, s=10, cmap='viridis')
        ax.set_xlabel('Dim 1')
        ax.set_ylabel('Dim 2')
        ax.set_zlabel("Dim 3")
        ax.set_title('HPC')
        # ax.axis('equal')
    #     cb_ax = fig.add_axes([0.95, 0.1, 0.02, 0.8])
    #     cbar = fig.colorbar(p3, cax=cb_ax, label = 'Dose')    

        # AV matrix HPC
        ax2 = fig.add_subplot(2,2,2)
        sns.heatmap(corr_AV_HPC, cmap = 'jet', ax=ax2, cbar_kws={'label': 'Correlation (r)'})
        ## lines to separate injections
        ax2.plot(np.ones(corr_AV_HPC.shape[0])*30, np.arange(0,corr_AV_HPC.shape[0]), '--', color = 'white')
        ax2.plot(np.arange(0,corr_AV_HPC.shape[0]), np.ones(corr_AV_HPC.shape[0])*30, '--', color = 'white')
        ax2.plot(np.ones(corr_AV_HPC.shape[0])*60, np.arange(0,corr_AV_HPC.shape[0]), '--', color = 'white')
        ax2.plot(np.arange(0,corr_AV_HPC.shape[0]), np.ones(corr_AV_HPC.shape[0])*60, '--', color = 'white')
        ax2.plot(np.ones(corr_AV_HPC.shape[0])*90, np.arange(0,corr_AV_HPC.shape[0]), '--', color = 'white')
        ax2.plot(np.arange(0,corr_AV_HPC.shape[0]), np.ones(corr_AV_HPC.shape[0])*90, '--', color = 'white')
        ax2.plot(np.ones(corr_AV_HPC.shape[0])*120, np.arange(0,corr_AV_HPC.shape[0]), '--', color = 'white')
        ax2.plot(np.arange(0,corr_AV_HPC.shape[0]), np.ones(corr_AV_HPC.shape[0])*120, '--', color = 'white')
        ax2.set_title('HPC')
        ax2.set_xlabel('Time [min]')
        ax2.set_ylabel('Time [min]')

        # isomap projection PFC
        ax3 = fig.add_subplot(2,2,3, projection = '3d')
        tmp_sim = []
        tmp_sim = ISO_all_PFC[irec_real].copy()
        p4 = ax3.scatter(tmp_sim[:, 0], tmp_sim[:, 1], tmp_sim[:, 2], c=dose_color, s=10, cmap='viridis')
        ax3.set_xlabel('Dim 1')
        ax3.set_ylabel('Dim 2')
        ax3.set_zlabel("Dim 3")
        ax3.set_title('PFC')

        # ax.axis('equal')
    #     cb_ax2 = fig.add_axes([0.95, 0.1, 0.02, 0.8])
    #     cbar2 = fig.colorbar(p4, cax=cb_ax2, label = 'Dose')       

        ## AV matrix PFC
        ax4 = fig.add_subplot(2,2,4)
        sns.heatmap(corr_AV_PFC, cmap = 'jet', ax= ax4, cbar_kws={'label': 'Correlation (r)'})
        ## lines to separate injections
        ax4.plot(np.ones(corr_AV_HPC.shape[0])*30, np.arange(0,corr_AV_HPC.shape[0]), '--', color = 'white')
        ax4.plot(np.arange(0,corr_AV_HPC.shape[0]), np.ones(corr_AV_HPC.shape[0])*30, '--', color = 'white')
        ax4.plot(np.ones(corr_AV_HPC.shape[0])*60, np.arange(0,corr_AV_HPC.shape[0]), '--', color = 'white')
        ax4.plot(np.arange(0,corr_AV_HPC.shape[0]), np.ones(corr_AV_HPC.shape[0])*60, '--', color = 'white')
        ax4.plot(np.ones(corr_AV_HPC.shape[0])*90, np.arange(0,corr_AV_HPC.shape[0]), '--', color = 'white')
        ax4.plot(np.arange(0,corr_AV_HPC.shape[0]), np.ones(corr_AV_HPC.shape[0])*90, '--', color = 'white')
        ax4.plot(np.ones(corr_AV_HPC.shape[0])*120, np.arange(0,corr_AV_HPC.shape[0]), '--', color = 'white')
        ax4.plot(np.arange(0,corr_AV_HPC.shape[0]), np.ones(corr_AV_HPC.shape[0])*120, '--', color = 'white')
        ax4.set_title('mPFC')
        ax4.set_xlabel('Time [min]')
        ax4.set_ylabel('Time [min]')       
        plt.tight_layout()
        plt.show()
        
        irec_real = irec_real + 1


## UMAP projection

In [ ]:
## units in [cells x time], note: isomap algorithm takes the input as time x cells!!

def UMAP_proj(behavior, units):
    
    import umap
    
    reducer = umap.UMAP(metric='euclidean', min_dist=0.3, n_components=2, n_neighbors=15)
    
    tmp_sim = []
    _inp_l = []
    time = []

    _inp_l = units

    Xl = np.sqrt(_inp_l.T)
    print(Xl.shape)
    
    embedding = []
    embedding = reducer.fit_transform(Xl)
    print(embedding.shape)
    
    tmp_sim = embedding.copy()
    
    fig = plt.figure(figsize=(6, 6))
    ax = fig.add_subplot(1,1,1)
    p3 = ax.scatter(tmp_sim[:, 0], tmp_sim[:, 1], c=behavior, s=10, cmap='viridis')
    ax.set_xlabel('Dim 1')
    ax.set_ylabel('Dim 2')
    # ax.axis('equal')
    cb_ax = fig.add_axes([0.95, 0.1, 0.02, 0.8])
    cbar = fig.colorbar(p3, cax=cb_ax, label = 'Dose')
    plt.show()

    behavior = []
    units = []

    return tmp_sim

In [ ]:
## UMAP projections
%matplotlib notebook

UMAP_all_HPC = []
UMAP_all_PFC = []
UMAP_all_HPCPFC = []

sampRate = 10
idx_plot = 0

for irec in range(0,1): #len(x)):
    
    if (drug_list[irec] == 'SAL') or (drug_list[irec] == 'RSK'):

        ksort_path = ksort_HPC_path_matched[irec]
        print('experiment',ksort_path)        

        if spikes_binned_all_HPC[irec].shape[0] > 20:

            if  spikes_binned_all_HPC[irec].shape[1] > 7201*sampRate:
                dose_color = []
                dose_color = np.hstack([np.ones(baseline), 2*np.ones(baseline), 3*np.ones(baseline),
                                       4*np.ones(baseline), 5*np.ones(spikes_binned_all_HPC[irec].shape[1]-dose3)])       

            if  spikes_binned_all_HPC[irec].shape[1] <= 7201*sampRate:
                dose_color = []
                dose_color = np.hstack([np.ones(baseline), 2*np.ones(baseline), 3*np.ones(baseline),
                                       4*np.ones(spikes_binned_all_HPC[irec].shape[1]-dose2)])    

            print('HPC')    
            tmp_UMAP = []
            tmp_UMAP,c1 = UMAP_proj(dose_color, spikes_binned_all_HPC[irec])
            UMAP_all_HPC.append(tmp_UMAP)
#             plt.savefig(r'G:\My Drive\Fenton_Lab\Ketamine_analysis\Manuscript\Umap_HPC' + str(idx_plot) + '.pdf', transparent = True)
#             plt.show()
#             plt.clf()
#             plt.close()  

            print('mPFC')        
            tmp_UMAP = []
            tmp_UMAP,c2 = UMAP_proj(dose_color, spikes_binned_all_PFC[irec])    
            UMAP_all_PFC.append(tmp_UMAP)    
#             plt.savefig(r'G:\My Drive\Fenton_Lab\Ketamine_analysis\Manuscript\UMAP_PFC' + str(idx_plot) + '.pdf', transparent = True)
#             plt.show()
#             plt.clf()
#             plt.close()           

            print('HPC+mPFC')        
            tmp_UMAP = []
            tmp_UMAP,c2 = UMAP_proj(dose_color, spikes_binned_HPC_PFC[irec])    
            UMAP_all_HPCPFC.append(tmp_UMAP)    
#             plt.savefig(r'G:\My Drive\Fenton_Lab\Ketamine_analysis\Manuscript\UMAP_HPC_PFC' + str(idx_plot) + '.pdf', transparent = True)
#             plt.show()
#             plt.clf()
#             plt.close()   

            idx_plot = idx_plot + 1
            